# Module Load

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy.stats import zscore
import seaborn as sns
from scipy.stats import linregress
from statsmodels.formula.api import mixedlm
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats
import numpy as np






# Functions


## Describe a column

In [ ]:
def describe_column(column_name):
    # Describe the column
    description = redcap[column_name].describe()
    print(description)
    
    # Count the number of NaN values in the column
    nan_count = redcap[column_name].isna().sum()
    print(f"Number of NaN values: {nan_count}")

## Basic stat

In [ ]:
def basic_stat(df):
    
    # Initialize the DataFrame with the specified columns
    results = pd.DataFrame(columns=['Pré-Chirurgie', '4 mois', '12 mois', '24 mois', 'Valeur P'])

    # Extend type_chx
    df['session'] = df['session'].astype(str)
        

    # Fill NaN values in type_chx column with the value from session 1 for the same id_participant
    df['type_chx'] = df.groupby('id_participant')['type_chx'].transform(lambda x: x.fillna(x.iloc[0]))
    # Fill NaN values in sexeF column with the value from session 1 for the same id_participant
    df['sexeF'] = df.groupby('id_participant')['sexeF'].transform(lambda x: x.fillna(x.iloc[0]))

    # Count the unique id_participant for session = 1,2,3,4
    unique_participants_per_session = df.groupby('session')['id_participant'].nunique()

    # Extract the number of unique participants for each session
    N_pre_chirurgie = unique_participants_per_session.get('1', 0)
    N_4_mois = unique_participants_per_session.get('2', 0)
    N_12_mois = unique_participants_per_session.get('3', 0)
    N_24_mois = unique_participants_per_session.get('4', 0)

    # Calculate the mean and standard deviation of age for each session
    mean_age_per_session = df.groupby('session')['age'].mean()
    std_age_per_session = df.groupby('session')['age'].std()

    # Extract the mean and standard deviation of age for each session
    mean_std_age_pre_chirurgie = f"{mean_age_per_session.get('1', 0):.2f} ± {std_age_per_session.get('1', 0):.2f}"
    mean_std_age_4_mois = f"{mean_age_per_session.get('2', 0):.2f} ± {std_age_per_session.get('2', 0):.2f}"
    mean_std_age_12_mois = f"{mean_age_per_session.get('3', 0):.2f} ± {std_age_per_session.get('3', 0):.2f}"
    mean_std_age_24_mois = f"{mean_age_per_session.get('4', 0):.2f} ± {std_age_per_session.get('4', 0):.2f}"

    # Count the number of sexeF = 1 per unique id_participant for each session
    sexeF_counts_per_session = df[df['sexeF'] == 1].drop_duplicates(subset=['session', 'id_participant']).groupby('session')['id_participant'].nunique()

    # Count the number of sexeF = 0 per unique id_participant for each session
    sexeM_counts_per_session = df[df['sexeF'] == 0].drop_duplicates(subset=['session', 'id_participant']).groupby('session')['id_participant'].nunique()

    # Create the variables for each session
    sexe_pre_chirurgie = {0: sexeM_counts_per_session.get('1', 0), 1: sexeF_counts_per_session.get('1', 0)}
    sexe_4_mois = {0: sexeM_counts_per_session.get('2', 0), 1: sexeF_counts_per_session.get('2', 0)}
    sexe_12_mois = {0: sexeM_counts_per_session.get('3', 0), 1: sexeF_counts_per_session.get('3', 0)}
    sexe_24_mois = {0: sexeM_counts_per_session.get('4', 0), 1: sexeF_counts_per_session.get('4', 0)}

    # Count the number of times type_chx = 1 per unique id_participant for each session
    gastrectomie_counts_per_session = df[df['type_chx'] == 1].drop_duplicates(subset=['session', 'id_participant']).groupby(['session', 'id_participant']).size().unstack(fill_value=0)

    gastrectomie_pre_chirurgie = gastrectomie_counts_per_session.loc['1'].sum() if '1' in gastrectomie_counts_per_session.index else 0
    gastrectomie_4_mois = gastrectomie_counts_per_session.loc['2'].sum() if '2' in gastrectomie_counts_per_session.index else 0
    gastrectomie_12_mois = gastrectomie_counts_per_session.loc['3'].sum() if '3' in gastrectomie_counts_per_session.index else 0
    gastrectomie_24_mois = gastrectomie_counts_per_session.loc['4'].sum() if '4' in gastrectomie_counts_per_session.index else 0

    # Count the number of times type_chx = 2 per unique id_participant for each session
    rygb_counts_per_session = df[df['type_chx'] == 2].drop_duplicates(subset=['session', 'id_participant']).groupby(['session', 'id_participant']).size().unstack(fill_value=0)

    rygb_pre_chirurgie = rygb_counts_per_session.loc['1'].sum() if '1' in rygb_counts_per_session.index else 0
    rygb_4_mois = rygb_counts_per_session.loc['2'].sum() if '2' in rygb_counts_per_session.index else 0
    rygb_12_mois = rygb_counts_per_session.loc['3'].sum() if '3' in rygb_counts_per_session.index else 0
    rygb_24_mois = rygb_counts_per_session.loc['4'].sum() if '4' in rygb_counts_per_session.index else 0

    # Count the number of times type_chx = 3 per unique id_participant for each session
    dbp_sd_counts_per_session = df[df['type_chx'] == 3].drop_duplicates(subset=['session', 'id_participant']).groupby(['session', 'id_participant']).size().unstack(fill_value=0)

    dbp_sd_pre_chirurgie = dbp_sd_counts_per_session.loc['1'].sum() if '1' in dbp_sd_counts_per_session.index else 0
    dbp_sd_4_mois = dbp_sd_counts_per_session.loc['2'].sum() if '2' in dbp_sd_counts_per_session.index else 0
    dbp_sd_12_mois = dbp_sd_counts_per_session.loc['3'].sum() if '3' in dbp_sd_counts_per_session.index else 0
    dbp_sd_24_mois = dbp_sd_counts_per_session.loc['4'].sum() if '4' in dbp_sd_counts_per_session.index else 0

    # Calculate the mean and standard deviation of imc_recherche for each session
    mean_imc_per_session = df.groupby('session')['imc_recherche_x'].mean()
    std_imc_per_session = df.groupby('session')['imc_recherche_x'].std()

    # Extract the mean and standard deviation of imc_recherche for each session
    mean_std_imc_pre_chirurgie = f"{mean_imc_per_session.get('1', 0):.2f} ± {std_imc_per_session.get('1', 0):.2f}"
    mean_std_imc_4_mois = f"{mean_imc_per_session.get('2', 0):.2f} ± {std_imc_per_session.get('2', 0):.2f}"
    mean_std_imc_12_mois = f"{mean_imc_per_session.get('3', 0):.2f} ± {std_imc_per_session.get('3', 0):.2f}"
    mean_std_imc_24_mois = f"{mean_imc_per_session.get('4', 0):.2f} ± {std_imc_per_session.get('4', 0):.2f}"

    # Calculate the mean and standard deviation of twl_recherche_x for each session
    mean_twl_per_session = df.groupby('session')['twl_recherche_x'].mean() * 100
    std_twl_per_session = df.groupby('session')['twl_recherche_x'].std() * 100

    # Extract the mean and standard deviation of twl_recherche_x for each session
    mean_std_twl_pre_chirurgie = f"{mean_twl_per_session.get('1', 0):.2f} ± {std_twl_per_session.get('1', 0):.2f}"
    mean_std_twl_4_mois = f"{mean_twl_per_session.get('2', 0):.2f} ± {std_twl_per_session.get('2', 0):.2f}"
    mean_std_twl_12_mois = f"{mean_twl_per_session.get('3', 0):.2f} ± {std_twl_per_session.get('3', 0):.2f}"
    mean_std_twl_24_mois = f"{mean_twl_per_session.get('4', 0):.2f} ± {std_twl_per_session.get('4', 0):.2f}"

    

    # Add the mean and standard deviation of age to the results DataFrame
    results.loc['N'] = [N_pre_chirurgie, N_4_mois, N_12_mois, N_24_mois, None]
    results.loc['Age (mean ± std)'] = [mean_std_age_pre_chirurgie, mean_std_age_4_mois, mean_std_age_12_mois, mean_std_age_24_mois, None]
    # Format the counts as "count of 0 : count of 1"
    format_sexe_counts = lambda counts: f"{counts.get(0, 0)} : {counts.get(1, 0)}"
    results.loc['Sexe (M:F)'] = [format_sexe_counts(sexe_pre_chirurgie), format_sexe_counts(sexe_4_mois), format_sexe_counts(sexe_12_mois), format_sexe_counts(sexe_24_mois), None]
    # Add the gastrectomie counts to the results DataFrame
    results.loc['Gastrectomie (n)'] = [gastrectomie_pre_chirurgie, gastrectomie_4_mois, gastrectomie_12_mois, gastrectomie_24_mois, None]
    # Add the RYGB counts to the results DataFrame
    results.loc['RYGB (n)'] = [rygb_pre_chirurgie, rygb_4_mois, rygb_12_mois, rygb_24_mois, None]

    # Add the DBP-SD counts to the results DataFrame
    results.loc['DBP-SD (n)'] = [dbp_sd_pre_chirurgie, dbp_sd_4_mois, dbp_sd_12_mois, dbp_sd_24_mois, None]

    # Add the mean and standard deviation of imc_recherche to the results DataFrame
    results.loc['IMC (kg/m2) (moy. ± et.)'] = [mean_std_imc_pre_chirurgie, mean_std_imc_4_mois, mean_std_imc_12_mois, mean_std_imc_24_mois, None]

    # Add the mean and standard deviation of twl_recherche_x to the results DataFrame
    results.loc['Pourcentage de perte de poids (%)'] = [mean_std_twl_pre_chirurgie, mean_std_twl_4_mois, mean_std_twl_12_mois, mean_std_twl_24_mois, None]

    return results


# Data load and prep

## Load Raw Participant file

In [ ]:
# Load the CSV file
file_path = '/path/to/your/data/directory/redcap_export_data.csv'
redcap = pd.read_csv(file_path)


## Prepare participant file for GI analysis

In [ ]:
redcap.rename(columns={'fteq38': 'tfeq38'}, inplace=True)

# Convert all string values in the dataframe to lowercase
redcap = redcap.applymap(lambda x: x.lower() if isinstance(x, str) else x)

# Find and deal with missing values in the 'id_recherchescan' column
nan_id_recherchescan = redcap[redcap['id_recherchescan'].isna()]['record_id'].unique()

# Update 'id_recherchescan' for specific 'record_id' values
# Example mappings (update with your specific IDs)
redcap.loc[redcap['record_id'] == 1, 'id_recherchescan'] = 'sub001'
redcap.loc[redcap['record_id'] == 6, 'id_recherchescan'] = 'sub002'
redcap.loc[redcap['record_id'] == 18, 'id_recherchescan'] = 'sub003'
redcap.loc[redcap['record_id'] == 22, 'id_recherchescan'] = 'sub004'
redcap.loc[redcap['record_id'] == 35, 'id_recherchescan'] = 'sub005'
redcap.loc[redcap['record_id'] == 45, 'id_recherchescan'] = 'sub006'

In [ ]:
# Save the redcap DataFrame to a CSV file
redcap.to_csv('processed_participant_data.csv', index=False)

## Prepare GI hormones dataframe

In [ ]:
redcap_GI = redcap.copy()
redcap_GI


In [ ]:
# Load the CSV file into a DataFrame
file_path = '/path/to/your/data/directory/hormones_data.csv'
hormones_GI_gutbrain = pd.read_csv(file_path)

# Create a new column 'id_participant' which is the lowercase of 'Identification Patient'
hormones_GI_gutbrain['id_participant'] = hormones_GI_gutbrain['Identification Patient'].str.lower()

In [ ]:
# Extract unique combinations of session and id_participant from redcap_GI
redcap_GI_unique = redcap_GI[['session', 'id_participant']].drop_duplicates()

# Extract unique combinations of session and id_participant from hormones_GI_gutbrain
hormones_GI_gutbrain_unique = hormones_GI_gutbrain[['session', 'id_participant']].drop_duplicates()

# Find unique combinations in redcap_GI that are not in hormones_GI_gutbrain
unique_combinations = redcap_GI_unique.merge(hormones_GI_gutbrain_unique, on=['session', 'id_participant'], how='left', indicator=True)
unique_combinations = unique_combinations[unique_combinations['_merge'] == 'left_only'].drop(columns=['_merge'])
unique_combinations.to_csv('missing_data_comparison.csv', index=False)
print(unique_combinations)

In [ ]:
# Assuming hormones_GI_gutbrain and redcap_GI are already defined as DataFrames
redcap_GI = pd.merge(hormones_GI_gutbrain, redcap_GI, on=['id_participant', 'session'])
redcap_GI
redcap_GI.to_csv('merged_hormone_data.csv', index=False)

## Load Raw Bid File

In [ ]:
# Load the file into a DataFrame
file_path = '/path/to/your/data/directory/bidding_behavior_data.csv'
bid_avg_hi_lo = pd.read_csv(file_path)
# Remove the 'sexe' column if it exists
if 'sexe' in bid_avg_hi_lo.columns:
    bid_avg_hi_lo = bid_avg_hi_lo.drop(columns=['sexe'])
# Extend type_chx for the same values of id_participant
bid_avg_hi_lo['type_chx'] = bid_avg_hi_lo.groupby('id_participant')['type_chx'].transform(lambda x: x.ffill().bfill())

In [ ]:
basic_stat(bid_avg_hi_lo)

In [ ]:
print(hormones_GI_gutbrain.columns)
print(bid_avg_hi_lo.columns)

In [ ]:
bid_avg_hi = bid_avg_hi_lo[bid_avg_hi_lo['calories'] != 'lo']
bid_avg_lo = bid_avg_hi_lo[bid_avg_hi_lo['calories'] != 'hi']

### Study the effect of surgery type on bid high

In [ ]:
plt.figure(figsize=(10, 6))

# Plotting the data
sns.lineplot(data=bid_avg_hi, x='session', y='bid', hue='type_chx', marker='o')

# Adding title and labels
plt.title('Bid as a function of Session for Different Surgery Types')
plt.xlabel('Session')
plt.ylabel('Bid')
plt.legend(title='Surgery Type')

# Display the plot
plt.show()

In [ ]:
# Define the formula
formula = 'bid ~ age + type_chx + session + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, bid_avg_hi, groups=bid_avg_hi['id_participant'], re_formula='1')
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")

In [ ]:
# Merge bid_avg_hi_lo with redcap_GI, prioritizing redcap_GI columns
df_GI_bid = pd.merge(hormones_GI_gutbrain, bid_avg_hi_lo, on=['id_participant', 'session'], how='left', suffixes=('', '_dup'))

# Drop duplicated columns from bid_avg_hi_lo
df_GI_bid = df_GI_bid.loc[:, ~df_GI_bid.columns.str.endswith('_dup')]

In [ ]:
df_GI_bid['Ghrelin_Totale'] = df_GI_bid['Ghrelin_Unacylated'] + df_GI_bid['Ghrelin_acylated']

In [ ]:
basic_stat(df_GI_bid)

In [ ]:
df_GI_bid

In [ ]:

columns_to_zscore = ['GLP_1_total', 'Leptin', 'PYY', 'Ghrelin_Unacylated', 'Ghrelin_acylated','Ghrelin_Totale']

df_GI_bid_no_outlier = df_GI_bid.copy()

for column in columns_to_zscore:
    df_GI_bid[column + '_zscore'] = zscore(df_GI_bid[column], nan_policy='omit')
df_GI_bid

zscore_columns = [column + '_zscore' for column in columns_to_zscore]

df_GI_bid_no_outlier = df_GI_bid[
    (df_GI_bid[zscore_columns] <= 3).all(axis=1) & 
    (df_GI_bid[zscore_columns] >= -3).all(axis=1)
]
df_GI_bid_no_outlier


In [ ]:
# Save the merged DataFrame to a CSV file
#df_GI_bid.to_csv('df_GI_bid.csv', index=False)


## Create df_GI_bid_hi_ac

In [ ]:
# Create a new df based on fasted state and bid for hi calorie foods

df_GI_bid_hi_ac = df_GI_bid.copy()
df_GI_bid_hi_ac = df_GI_bid_hi_ac[(df_GI_bid_hi_ac['fed_state'] == 'ac') & (df_GI_bid_hi_ac['calories'] == 'hi')]



#df_no_PYY = df_GI_bid_hi_ac[df_GI_bid_hi_ac['PYY'].isna()][['id_participant', 'session']].drop_duplicates()
#df_no_PYY

#df_no_Ghrelin_Unacylated = df_GI_bid_hi_ac[df_GI_bid_hi_ac['Ghrelin_Unacylated'].isna()][['id_participant', 'session']].drop_duplicates()
#df_no_Ghrelin_Unacylated

#df_no_Ghrelin_acylated = df_GI_bid_hi_ac[df_GI_bid_hi_ac['Ghrelin_acylated'].isna()][['id_participant', 'session']].drop_duplicates()
#df_no_Ghrelin_acylated



#df_GI_bid_hi_ac_PYY = df_GI_bid_hi_ac.dropna(subset=['PYY'])
#df_GI_bid_hi_ac_PYY

#df_GI_bid_hi_ac_Ghrelin_Unacylated = df_GI_bid_hi_ac.dropna(subset=['Ghrelin_Unacylated'])
#df_GI_bid_hi_ac_Ghrelin_Unacylated

#df_GI_bid_hi_ac_Ghrelin_acylated = df_GI_bid_hi_ac.dropna(subset=['Ghrelin_acylated'])
#df_GI_bid_hi_ac_Ghrelin_acylated

#df_no_Ghrelin_Totale = df_GI_bid_hi_ac[df_GI_bid_hi_ac['Ghrelin_Totale'].isna()][['id_participant', 'session']].drop_duplicates()
#df_no_Ghrelin_Totale

#df_GI_bid_hi_ac_Ghrelin_Totale = df_GI_bid_hi_ac.dropna(subset=['Ghrelin_Totale'])
#df_GI_bid_hi_ac_Ghrelin_Totale


### Create DFs for GLP1 analysis AC

In [ ]:
# Find participants and sessions where GLP1 is not recorded (AC, HI)
df_no_GLP_1_total = df_GI_bid_hi_ac[df_GI_bid_hi_ac['GLP_1_total'].isna()][['id_participant', 'session']].drop_duplicates()
#print(df_no_GLP_1_total)

# Create a DF where there are no Nans for GLP1 AC HI
df_GI_bid_hi_ac_GLP1 = df_GI_bid_hi_ac.dropna(subset=['GLP_1_total'])

# Calculate z-scores for GLP_1_total and session
df_GI_bid_hi_ac_GLP1['GLP_1_total_zscore_session'] = df_GI_bid_hi_ac_GLP1.groupby('session')['GLP_1_total'].transform(lambda x: (x - x.mean()) / x.std())
df_GI_bid_hi_ac_GLP1['GLP_1_total_zscore'] = zscore(df_GI_bid_hi_ac_GLP1['GLP_1_total'])
#print(df_GI_bid_hi_ac_GLP1['GLP_1_total_zscore_session'])

# Print GLP_1_total_zscore_session and GLP_1_total_zscore
#print(df_GI_bid_hi_ac_GLP1[['GLP_1_total_zscore_session', 'GLP_1_total_zscore']])

#print(basic_stat(df_GI_bid_hi_ac_GLP1))

# Find outliers based on GLP1 zscore
outliers_GLP1 = df_GI_bid_hi_ac_GLP1[(df_GI_bid_hi_ac_GLP1['GLP_1_total_zscore'] > 3) | (df_GI_bid_hi_ac_GLP1['GLP_1_total_zscore'] < -3)]
#print(outliers_GLP1[['id_participant', 'session']])

# Find outliers based on GLP1 zscore per session
outliers_GLP1_session = df_GI_bid_hi_ac_GLP1[(df_GI_bid_hi_ac_GLP1['GLP_1_total_zscore_session'] > 3) | (df_GI_bid_hi_ac_GLP1['GLP_1_total_zscore_session'] < -3)]
print(outliers_GLP1_session[['id_participant', 'session','GLP_1_total']])

# Remove outliers based on z-score threshold of 3
df_GI_bid_hi_ac_GLP1_no_outliers = df_GI_bid_hi_ac_GLP1[df_GI_bid_hi_ac_GLP1['GLP_1_total_zscore'].abs() <= 3]
#print(basic_stat(df_GI_bid_hi_ac_GLP1_no_outliers))

# Remove outliers based on z-score threshold of 3 for session
df_GI_bid_hi_ac_GLP1_no_outliers_session = df_GI_bid_hi_ac_GLP1[df_GI_bid_hi_ac_GLP1['GLP_1_total_zscore_session'].abs() <= 3]
#print(basic_stat(df_GI_bid_hi_ac_GLP1_no_outliers_session))


### Create dfs for Leptin analysis AC

In [ ]:
# Find participants and sessions where Leptin is not recorded (AC, HI)
df_no_Leptin = df_GI_bid_hi_ac[df_GI_bid_hi_ac['Leptin'].isna()][['id_participant', 'session']].drop_duplicates()
#print(df_no_Leptin)

# Create a DF where there are no Nans for Leptin AC HI
df_GI_bid_hi_ac_Leptin = df_GI_bid_hi_ac.dropna(subset=['Leptin'])

# Calculate z-scores for Leptin and session
df_GI_bid_hi_ac_Leptin['Leptin_zscore_session'] = df_GI_bid_hi_ac_Leptin.groupby('session')['Leptin'].transform(lambda x: (x - x.mean()) / x.std())
df_GI_bid_hi_ac_Leptin['Leptin_zscore'] = zscore(df_GI_bid_hi_ac_Leptin['Leptin'])

# Print Leptin_zscore_session and Leptin_zscore
#print(df_GI_bid_hi_ac_Leptin[['Leptin_zscore_session', 'Leptin_zscore']])

#print(basic_stat(df_GI_bid_hi_ac_Leptin))

# Find outliers based on Leptin zscore
outliers_Leptin = df_GI_bid_hi_ac_Leptin[(df_GI_bid_hi_ac_Leptin['Leptin_zscore'] > 3) | (df_GI_bid_hi_ac_Leptin['Leptin_zscore'] < -3)]
#print(outliers_Leptin[['id_participant', 'session']])

# Find outliers based on Leptin zscore per session
outliers_Leptin_session = df_GI_bid_hi_ac_Leptin[(df_GI_bid_hi_ac_Leptin['Leptin_zscore_session'] > 3) | (df_GI_bid_hi_ac_Leptin['Leptin_zscore_session'] < -3)]
#print(outliers_Leptin_session[['id_participant', 'session']])

# Remove outliers based on z-score threshold of 3
df_GI_bid_hi_ac_Leptin_no_outliers = df_GI_bid_hi_ac_Leptin[df_GI_bid_hi_ac_Leptin['Leptin_zscore'].abs() <= 3]
#print(basic_stat(df_GI_bid_hi_ac_Leptin_no_outliers))

# Remove outliers based on z-score threshold of 3 for session
df_GI_bid_hi_ac_Leptin_no_outliers_session = df_GI_bid_hi_ac_Leptin[df_GI_bid_hi_ac_Leptin['Leptin_zscore_session'].abs() <= 3]
#print(basic_stat(df_GI_bid_hi_ac_Leptin_no_outliers_session))

### Create dfs for Ghrelin Unacetylated AC

In [ ]:
# Find participants and sessions where Ghrelin_Unacylated is not recorded (AC, HI)
df_no_Ghrelin_Unacylated = df_GI_bid_hi_ac[df_GI_bid_hi_ac['Ghrelin_Unacylated'].isna()][['id_participant', 'session']].drop_duplicates()
#print(df_no_Ghrelin_Unacylated)

# Create a DF where there are no Nans for Ghrelin_Unacylated AC HI
df_GI_bid_hi_ac_Ghrelin_Unacylated = df_GI_bid_hi_ac.dropna(subset=['Ghrelin_Unacylated'])

# Calculate z-scores for Ghrelin_Unacylated and session
df_GI_bid_hi_ac_Ghrelin_Unacylated['Ghrelin_Unacylated_zscore_session'] = df_GI_bid_hi_ac_Ghrelin_Unacylated.groupby('session')['Ghrelin_Unacylated'].transform(lambda x: (x - x.mean()) / x.std())
df_GI_bid_hi_ac_Ghrelin_Unacylated['Ghrelin_Unacylated_zscore'] = zscore(df_GI_bid_hi_ac_Ghrelin_Unacylated['Ghrelin_Unacylated'])
#print(df_GI_bid_hi_ac_Ghrelin_Unacylated['Ghrelin_Unacylated_zscore_session'])

# Print Ghrelin_Unacylated_zscore_session and Ghrelin_Unacylated_zscore
#print(df_GI_bid_hi_ac_Ghrelin_Unacylated[['Ghrelin_Unacylated_zscore_session', 'Ghrelin_Unacylated_zscore']])

#print(basic_stat(df_GI_bid_hi_ac_Ghrelin_Unacylated))

# Find outliers based on Ghrelin_Unacylated zscore
outliers_Ghrelin_Unacylated = df_GI_bid_hi_ac_Ghrelin_Unacylated[(df_GI_bid_hi_ac_Ghrelin_Unacylated['Ghrelin_Unacylated_zscore'] > 3) | (df_GI_bid_hi_ac_Ghrelin_Unacylated['Ghrelin_Unacylated_zscore'] < -3)]
#print(outliers_Ghrelin_Unacylated[['id_participant', 'session']])

# Find outliers based on Ghrelin_Unacylated zscore per session
outliers_Ghrelin_Unacylated_session = df_GI_bid_hi_ac_Ghrelin_Unacylated[(df_GI_bid_hi_ac_Ghrelin_Unacylated['Ghrelin_Unacylated_zscore_session'] > 3) | (df_GI_bid_hi_ac_Ghrelin_Unacylated['Ghrelin_Unacylated_zscore_session'] < -3)]
#print(outliers_Ghrelin_Unacylated_session[['id_participant', 'session']])

# Remove outliers based on z-score threshold of 3
df_GI_bid_hi_ac_Ghrelin_Unacylated_no_outliers = df_GI_bid_hi_ac_Ghrelin_Unacylated[df_GI_bid_hi_ac_Ghrelin_Unacylated['Ghrelin_Unacylated_zscore'].abs() <= 3]
#print(basic_stat(df_GI_bid_hi_ac_Ghrelin_Unacylated_no_outliers))

# Remove outliers based on z-score threshold of 3 for session
df_GI_bid_hi_ac_Ghrelin_Unacylated_no_outliers_session = df_GI_bid_hi_ac_Ghrelin_Unacylated[df_GI_bid_hi_ac_Ghrelin_Unacylated['Ghrelin_Unacylated_zscore_session'].abs() <= 3]
#print(basic_stat(df_GI_bid_hi_ac_Ghrelin_Unacylated_no_outliers_session))

### Create dfs Ghrelin Acetylated AC

In [ ]:
# Find participants and sessions where Ghrelin_acylated is not recorded (AC, HI)
df_no_Ghrelin_acylated = df_GI_bid_hi_ac[df_GI_bid_hi_ac['Ghrelin_acylated'].isna()][['id_participant', 'session']].drop_duplicates()
#print(df_no_Ghrelin_acylated)

# Create a DF where there are no Nans for Ghrelin_acylated AC HI
df_GI_bid_hi_ac_Ghrelin_acylated = df_GI_bid_hi_ac.dropna(subset=['Ghrelin_acylated'])

# Calculate z-scores for Ghrelin_acylated and session
df_GI_bid_hi_ac_Ghrelin_acylated['Ghrelin_acylated_zscore_session'] = df_GI_bid_hi_ac_Ghrelin_acylated.groupby('session')['Ghrelin_acylated'].transform(lambda x: (x - x.mean()) / x.std())
df_GI_bid_hi_ac_Ghrelin_acylated['Ghrelin_acylated_zscore'] = zscore(df_GI_bid_hi_ac_Ghrelin_acylated['Ghrelin_acylated'])
#print(df_GI_bid_hi_ac_Ghrelin_acylated['Ghrelin_acylated_zscore_session'])

# Print Ghrelin_acylated_zscore_session and Ghrelin_acylated_zscore
#print(df_GI_bid_hi_ac_Ghrelin_acylated[['Ghrelin_acylated_zscore_session', 'Ghrelin_acylated_zscore']])

#print(basic_stat(df_GI_bid_hi_ac_Ghrelin_acylated))

# Find outliers based on Ghrelin_acylated zscore
outliers_Ghrelin_acylated = df_GI_bid_hi_ac_Ghrelin_acylated[(df_GI_bid_hi_ac_Ghrelin_acylated['Ghrelin_acylated_zscore'] > 3) | (df_GI_bid_hi_ac_Ghrelin_acylated['Ghrelin_acylated_zscore'] < -3)]
#print(outliers_Ghrelin_acylated[['id_participant', 'session']])

# Find outliers based on Ghrelin_acylated zscore per session
outliers_Ghrelin_acylated_session = df_GI_bid_hi_ac_Ghrelin_acylated[(df_GI_bid_hi_ac_Ghrelin_acylated['Ghrelin_acylated_zscore_session'] > 3) | (df_GI_bid_hi_ac_Ghrelin_acylated['Ghrelin_acylated_zscore_session'] < -3)]
#print(outliers_Ghrelin_acylated_session[['id_participant', 'session']])

# Remove outliers based on z-score threshold of 3
df_GI_bid_hi_ac_Ghrelin_acylated_no_outliers = df_GI_bid_hi_ac_Ghrelin_acylated[df_GI_bid_hi_ac_Ghrelin_acylated['Ghrelin_acylated_zscore'].abs() <= 3]
#print(basic_stat(df_GI_bid_hi_ac_Ghrelin_acylated_no_outliers))

# Remove outliers based on z-score threshold of 3 for session
df_GI_bid_hi_ac_Ghrelin_acylated_no_outliers_session = df_GI_bid_hi_ac_Ghrelin_acylated[df_GI_bid_hi_ac_Ghrelin_acylated['Ghrelin_acylated_zscore_session'].abs() <= 3]
#print(basic_stat(df_GI_bid_hi_ac_Ghrelin_acylated_no_outliers_session))

### Create dfs PYY AC

In [ ]:
# Find participants and sessions where PYY is not recorded (AC, HI)
df_no_PYY = df_GI_bid_hi_ac[df_GI_bid_hi_ac['PYY'].isna()][['id_participant', 'session']].drop_duplicates()
#print(df_no_PYY)

# Create a DF where there are no Nans for PYY AC HI
df_GI_bid_hi_ac_PYY = df_GI_bid_hi_ac.dropna(subset=['PYY'])

# Calculate z-scores for PYY and session
df_GI_bid_hi_ac_PYY['PYY_zscore_session'] = df_GI_bid_hi_ac_PYY.groupby('session')['PYY'].transform(lambda x: (x - x.mean()) / x.std())
df_GI_bid_hi_ac_PYY['PYY_zscore'] = zscore(df_GI_bid_hi_ac_PYY['PYY'])
#print(df_GI_bid_hi_ac_PYY['PYY_zscore_session'])

# Print PYY_zscore_session and PYY_zscore
#print(df_GI_bid_hi_ac_PYY[['PYY_zscore_session', 'PYY_zscore']])

#print(basic_stat(df_GI_bid_hi_ac_PYY))

# Find outliers based on PYY zscore
outliers_PYY = df_GI_bid_hi_ac_PYY[(df_GI_bid_hi_ac_PYY['PYY_zscore'] > 3) | (df_GI_bid_hi_ac_PYY['PYY_zscore'] < -3)]
#print(outliers_PYY[['id_participant', 'session']])

# Find outliers based on PYY zscore per session
outliers_PYY_session = df_GI_bid_hi_ac_PYY[(df_GI_bid_hi_ac_PYY['PYY_zscore_session'] > 3) | (df_GI_bid_hi_ac_PYY['PYY_zscore_session'] < -3)]
#print(outliers_PYY_session[['id_participant', 'session']])

# Remove outliers based on z-score threshold of 3
df_GI_bid_hi_ac_PYY_no_outliers = df_GI_bid_hi_ac_PYY[df_GI_bid_hi_ac_PYY['PYY_zscore'].abs() <= 3]
#print(basic_stat(df_GI_bid_hi_ac_PYY_no_outliers))

# Remove outliers based on z-score threshold of 3 for session
df_GI_bid_hi_ac_PYY_no_outliers_session = df_GI_bid_hi_ac_PYY[df_GI_bid_hi_ac_PYY['PYY_zscore_session'].abs() <= 3]
#print(basic_stat(df_GI_bid_hi_ac_PYY_no_outliers_session))

### Create dfs for ghreline Total AC

In [ ]:
# Find participants and sessions where Ghrelin_Totale is not recorded (AC, HI)
df_no_Ghrelin_Totale = df_GI_bid_hi_ac[df_GI_bid_hi_ac['Ghrelin_Totale'].isna()][['id_participant', 'session']].drop_duplicates()
#print(df_no_Ghrelin_Totale)

# Create a DF where there are no Nans for Ghrelin_Totale AC HI
df_GI_bid_hi_ac_Ghrelin_Totale = df_GI_bid_hi_ac.dropna(subset=['Ghrelin_Totale'])

# Calculate z-scores for Ghrelin_Totale and session
df_GI_bid_hi_ac_Ghrelin_Totale['Ghrelin_Totale_zscore_session'] = df_GI_bid_hi_ac_Ghrelin_Totale.groupby('session')['Ghrelin_Totale'].transform(lambda x: (x - x.mean()) / x.std())
df_GI_bid_hi_ac_Ghrelin_Totale['Ghrelin_Totale_zscore'] = zscore(df_GI_bid_hi_ac_Ghrelin_Totale['Ghrelin_Totale'])
df_GI_bid_hi_ac_Ghrelin_Totale['log_Ghrelin_Totale'] = np.log(df_GI_bid_hi_ac_Ghrelin_Totale['Ghrelin_Totale'])

#print(df_GI_bid_hi_ac_Ghrelin_Totale['Ghrelin_Totale_zscore_session'])
# Save the DataFrame to a CSV file
df_GI_bid_hi_ac_Ghrelin_Totale.to_csv('df_GI_bid_hi_ac_Ghrelin_Totale.csv', index=False)

# Print Ghrelin_Totale_zscore_session and Ghrelin_Totale_zscore
#print(df_GI_bid_hi_ac_Ghrelin_Totale[['Ghrelin_Totale_zscore_session', 'Ghrelin_Totale_zscore']])

#print(basic_stat(df_GI_bid_hi_ac_Ghrelin_Totale))

# Find outliers based on Ghrelin_Totale zscore
outliers_Ghrelin_Totale = df_GI_bid_hi_ac_Ghrelin_Totale[(df_GI_bid_hi_ac_Ghrelin_Totale['Ghrelin_Totale_zscore'] > 3) | (df_GI_bid_hi_ac_Ghrelin_Totale['Ghrelin_Totale_zscore'] < -3)]
print(outliers_Ghrelin_Totale[['id_participant', 'session']])

# Find outliers based on Ghrelin_Totale zscore per session
outliers_Ghrelin_Totale_session = df_GI_bid_hi_ac_Ghrelin_Totale[(df_GI_bid_hi_ac_Ghrelin_Totale['Ghrelin_Totale_zscore_session'] > 3) | (df_GI_bid_hi_ac_Ghrelin_Totale['Ghrelin_Totale_zscore_session'] < -3)]
print(outliers_Ghrelin_Totale_session[['id_participant', 'session']])

# Remove outliers based on z-score threshold of 3
df_GI_bid_hi_ac_Ghrelin_Totale_no_outliers = df_GI_bid_hi_ac_Ghrelin_Totale[df_GI_bid_hi_ac_Ghrelin_Totale['Ghrelin_Totale_zscore'].abs() <= 3]
#print(basic_stat(df_GI_bid_hi_ac_Ghrelin_Totale_no_outliers))

# Remove outliers based on z-score threshold of 3 for session
df_GI_bid_hi_ac_Ghrelin_Totale_no_outliers_session = df_GI_bid_hi_ac_Ghrelin_Totale[df_GI_bid_hi_ac_Ghrelin_Totale['Ghrelin_Totale_zscore_session'].abs() <= 3]
#print(basic_stat(df_GI_bid_hi_ac_Ghrelin_Totale_no_outliers_session))



## Create df_GI_bid_hi_ac_no_outlier

In [ ]:
df_GI_bid_hi_ac_no_outlier = df_GI_bid_no_outlier.copy()
df_GI_bid_hi_ac_no_outlier = df_GI_bid_hi_ac_no_outlier[(df_GI_bid_hi_ac_no_outlier['fed_state'] == 'ac') & (df_GI_bid_hi_ac['calories'] == 'hi')]


## Create df_GI_bid_hi_pp

In [ ]:
df_GI_bid_hi_pp = df_GI_bid.copy()
df_GI_bid_hi_pp = df_GI_bid_hi_pp[(df_GI_bid_hi_pp['fed_state'] == 'pp') & (df_GI_bid_hi_pp['calories'] == 'hi')]


### Create dfs GLP1 PP

In [ ]:
# Find participants and sessions where GLP1 is not recorded (PP)
df_no_GLP_1_total_pp = df_GI_bid_hi_pp[df_GI_bid_hi_pp['GLP_1_total'].isna()][['id_participant', 'session']].drop_duplicates()
#print(df_no_GLP_1_total_pp)

# Create a DF where there are no Nans for GLP1 PP
df_GI_bid_hi_pp_GLP1 = df_GI_bid_hi_pp.dropna(subset=['GLP_1_total'])

# Calculate z-scores for GLP_1_total and session
df_GI_bid_hi_pp_GLP1['GLP_1_total_zscore_session'] = df_GI_bid_hi_pp_GLP1.groupby('session')['GLP_1_total'].transform(lambda x: (x - x.mean()) / x.std())
df_GI_bid_hi_pp_GLP1['GLP_1_total_zscore'] = zscore(df_GI_bid_hi_pp_GLP1['GLP_1_total'])
#print(df_GI_bid_hi_pp_GLP1['GLP_1_total_zscore_session'])

# Print GLP_1_total_zscore_session and GLP_1_total_zscore
#print(df_GI_bid_hi_pp_GLP1[['GLP_1_total_zscore_session', 'GLP_1_total_zscore']])

#print(basic_stat(df_GI_bid_hi_pp_GLP1))

# Find outliers based on GLP1 zscore
outliers_GLP1_pp = df_GI_bid_hi_pp_GLP1[(df_GI_bid_hi_pp_GLP1['GLP_1_total_zscore'] > 3) | (df_GI_bid_hi_pp_GLP1['GLP_1_total_zscore'] < -3)]
print(outliers_GLP1_pp[['id_participant', 'session']])

# Find outliers based on GLP1 zscore per session
outliers_GLP1_session_pp = df_GI_bid_hi_pp_GLP1[(df_GI_bid_hi_pp_GLP1['GLP_1_total_zscore_session'] > 3) | (df_GI_bid_hi_pp_GLP1['GLP_1_total_zscore_session'] < -3)]
print(outliers_GLP1_session_pp[['id_participant', 'session']])

# Remove outliers based on z-score threshold of 3
df_GI_bid_hi_pp_GLP1_no_outliers = df_GI_bid_hi_pp_GLP1[df_GI_bid_hi_pp_GLP1['GLP_1_total_zscore'].abs() <= 3]
#print(basic_stat(df_GI_bid_hi_pp_GLP1_no_outliers))

# Remove outliers based on z-score threshold of 3 for session
df_GI_bid_hi_pp_GLP1_no_outliers_session = df_GI_bid_hi_pp_GLP1[df_GI_bid_hi_pp_GLP1['GLP_1_total_zscore_session'].abs() <= 3]
#print(basic_stat(df_GI_bid_hi_pp_GLP1_no_outliers_session))

# Create new DataFrames for each session
df_GI_bid_hi_pp_GLP1_ses_1 = df_GI_bid_hi_pp_GLP1[df_GI_bid_hi_pp_GLP1['session'] == '1']
df_GI_bid_hi_pp_GLP1_ses_2 = df_GI_bid_hi_pp_GLP1[df_GI_bid_hi_pp_GLP1['session'] == '2']
df_GI_bid_hi_pp_GLP1_ses_3 = df_GI_bid_hi_pp_GLP1[df_GI_bid_hi_pp_GLP1['session'] == '3']
df_GI_bid_hi_pp_GLP1_ses_4 = df_GI_bid_hi_pp_GLP1[df_GI_bid_hi_pp_GLP1['session'] == '4']

# Create new DataFrames for each session based on df_GI_bid_hi_pp_GLP1_no_outliers_session
df_GI_bid_hi_pp_GLP1_no_outliers_session_ses_1 = df_GI_bid_hi_pp_GLP1_no_outliers_session[df_GI_bid_hi_pp_GLP1_no_outliers_session['session'] == '1']
df_GI_bid_hi_pp_GLP1_no_outliers_session_ses_2 = df_GI_bid_hi_pp_GLP1_no_outliers_session[df_GI_bid_hi_pp_GLP1_no_outliers_session['session'] == '2']
df_GI_bid_hi_pp_GLP1_no_outliers_session_ses_3 = df_GI_bid_hi_pp_GLP1_no_outliers_session[df_GI_bid_hi_pp_GLP1_no_outliers_session['session'] == '3']
df_GI_bid_hi_pp_GLP1_no_outliers_session_ses_4 = df_GI_bid_hi_pp_GLP1_no_outliers_session[df_GI_bid_hi_pp_GLP1_no_outliers_session['session'] == '4']



### Create DFs Leptin PP

In [ ]:
# Find participants and sessions where Leptin is not recorded (PP)
df_no_Leptin_pp = df_GI_bid_hi_pp[df_GI_bid_hi_pp['Leptin'].isna()][['id_participant', 'session']].drop_duplicates()
print(df_no_Leptin_pp)

# Create a DF where there are no Nans for Leptin PP
df_GI_bid_hi_pp_Leptin = df_GI_bid_hi_pp.dropna(subset=['Leptin'])

# Calculate z-scores for Leptin and session
df_GI_bid_hi_pp_Leptin['Leptin_zscore_session'] = df_GI_bid_hi_pp_Leptin.groupby('session')['Leptin'].transform(lambda x: (x - x.mean()) / x.std())
df_GI_bid_hi_pp_Leptin['Leptin_zscore'] = zscore(df_GI_bid_hi_pp_Leptin['Leptin'])
#print(df_GI_bid_hi_pp_Leptin['Leptin_zscore_session'])

# Print Leptin_zscore_session and Leptin_zscore
#print(df_GI_bid_hi_pp_Leptin[['Leptin_zscore_session', 'Leptin_zscore']])

#print(basic_stat(df_GI_bid_hi_pp_Leptin))

# Find outliers based on Leptin zscore
outliers_Leptin_pp = df_GI_bid_hi_pp_Leptin[(df_GI_bid_hi_pp_Leptin['Leptin_zscore'] > 3) | (df_GI_bid_hi_pp_Leptin['Leptin_zscore'] < -3)]
print(outliers_Leptin_pp[['id_participant', 'session']])

# Find outliers based on Leptin zscore per session
outliers_Leptin_session_pp = df_GI_bid_hi_pp_Leptin[(df_GI_bid_hi_pp_Leptin['Leptin_zscore_session'] > 3) | (df_GI_bid_hi_pp_Leptin['Leptin_zscore_session'] < -3)]
print(outliers_Leptin_session_pp[['id_participant', 'session']])

# Remove outliers based on z-score threshold of 3
df_GI_bid_hi_pp_Leptin_no_outliers = df_GI_bid_hi_pp_Leptin[df_GI_bid_hi_pp_Leptin['Leptin_zscore'].abs() <= 3]
#print(basic_stat(df_GI_bid_hi_pp_Leptin_no_outliers))

# Remove outliers based on z-score threshold of 3 for session
df_GI_bid_hi_pp_Leptin_no_outliers_session = df_GI_bid_hi_pp_Leptin[df_GI_bid_hi_pp_Leptin['Leptin_zscore_session'].abs() <= 3]
#print(basic_stat(df_GI_bid_hi_pp_Leptin_no_outliers_session))

### Create DFs for Ghrelin Unacetylated PP

In [ ]:
# Find participants and sessions where Ghrelin_Unacylated is not recorded (PP)
df_no_Ghrelin_Unacylated_pp = df_GI_bid_hi_pp[df_GI_bid_hi_pp['Ghrelin_Unacylated'].isna()][['id_participant', 'session']].drop_duplicates()
#print(df_no_Ghrelin_Unacylated_pp)

# Create a DF where there are no Nans for Ghrelin_Unacylated PP
df_GI_bid_hi_pp_Ghrelin_Unacylated = df_GI_bid_hi_pp.dropna(subset=['Ghrelin_Unacylated'])

# Calculate z-scores for Ghrelin_Unacylated and session
df_GI_bid_hi_pp_Ghrelin_Unacylated['Ghrelin_Unacylated_zscore_session'] = df_GI_bid_hi_pp_Ghrelin_Unacylated.groupby('session')['Ghrelin_Unacylated'].transform(lambda x: (x - x.mean()) / x.std())
df_GI_bid_hi_pp_Ghrelin_Unacylated['Ghrelin_Unacylated_zscore'] = zscore(df_GI_bid_hi_pp_Ghrelin_Unacylated['Ghrelin_Unacylated'])
#print(df_GI_bid_hi_pp_Ghrelin_Unacylated['Ghrelin_Unacylated_zscore_session'])

# Print Ghrelin_Unacylated_zscore_session and Ghrelin_Unacylated_zscore
#print(df_GI_bid_hi_pp_Ghrelin_Unacylated[['Ghrelin_Unacylated_zscore_session', 'Ghrelin_Unacylated_zscore']])

#print(basic_stat(df_GI_bid_hi_pp_Ghrelin_Unacylated))

# Find outliers based on Ghrelin_Unacylated zscore
outliers_Ghrelin_Unacylated_pp = df_GI_bid_hi_pp_Ghrelin_Unacylated[(df_GI_bid_hi_pp_Ghrelin_Unacylated['Ghrelin_Unacylated_zscore'] > 3) | (df_GI_bid_hi_pp_Ghrelin_Unacylated['Ghrelin_Unacylated_zscore'] < -3)]
#print(outliers_Ghrelin_Unacylated_pp[['id_participant', 'session']])

# Find outliers based on Ghrelin_Unacylated zscore per session
outliers_Ghrelin_Unacylated_session_pp = df_GI_bid_hi_pp_Ghrelin_Unacylated[(df_GI_bid_hi_pp_Ghrelin_Unacylated['Ghrelin_Unacylated_zscore_session'] > 3) | (df_GI_bid_hi_pp_Ghrelin_Unacylated['Ghrelin_Unacylated_zscore_session'] < -3)]
#print(outliers_Ghrelin_Unacylated_session_pp[['id_participant', 'session']])

# Remove outliers based on z-score threshold of 3
df_GI_bid_hi_pp_Ghrelin_Unacylated_no_outliers = df_GI_bid_hi_pp_Ghrelin_Unacylated[df_GI_bid_hi_pp_Ghrelin_Unacylated['Ghrelin_Unacylated_zscore'].abs() <= 3]
#print(basic_stat(df_GI_bid_hi_pp_Ghrelin_Unacylated_no_outliers))

# Remove outliers based on z-score threshold of 3 for session
df_GI_bid_hi_pp_Ghrelin_Unacylated_no_outliers_session = df_GI_bid_hi_pp_Ghrelin_Unacylated[df_GI_bid_hi_pp_Ghrelin_Unacylated['Ghrelin_Unacylated_zscore_session'].abs() <= 3]
#print(basic_stat(df_GI_bid_hi_pp_Ghrelin_Unacylated_no_outliers_session))

### Create DFs for Ghrelin ACetylayed PP

In [ ]:
# Find participants and sessions where Ghrelin_acylated is not recorded (PP)
df_no_Ghrelin_acylated_pp = df_GI_bid_hi_pp[df_GI_bid_hi_pp['Ghrelin_acylated'].isna()][['id_participant', 'session']].drop_duplicates()
print(df_no_Ghrelin_acylated_pp)

# Create a DF where there are no Nans for Ghrelin_acylated PP
df_GI_bid_hi_pp_Ghrelin_acylated = df_GI_bid_hi_pp.dropna(subset=['Ghrelin_acylated'])

# Calculate z-scores for Ghrelin_acylated and session
df_GI_bid_hi_pp_Ghrelin_acylated['Ghrelin_acylated_zscore_session'] = df_GI_bid_hi_pp_Ghrelin_acylated.groupby('session')['Ghrelin_acylated'].transform(lambda x: (x - x.mean()) / x.std())
df_GI_bid_hi_pp_Ghrelin_acylated['Ghrelin_acylated_zscore'] = zscore(df_GI_bid_hi_pp_Ghrelin_acylated['Ghrelin_acylated'])
#print(df_GI_bid_hi_pp_Ghrelin_acylated['Ghrelin_acylated_zscore_session'])

# Print Ghrelin_acylated_zscore_session and Ghrelin_acylated_zscore
#print(df_GI_bid_hi_pp_Ghrelin_acylated[['Ghrelin_acylated_zscore_session', 'Ghrelin_acylated_zscore']])

#print(basic_stat(df_GI_bid_hi_pp_Ghrelin_acylated))

# Find outliers based on Ghrelin_acylated zscore
outliers_Ghrelin_acylated_pp = df_GI_bid_hi_pp_Ghrelin_acylated[(df_GI_bid_hi_pp_Ghrelin_acylated['Ghrelin_acylated_zscore'] > 3) | (df_GI_bid_hi_pp_Ghrelin_acylated['Ghrelin_acylated_zscore'] < -3)]
print(outliers_Ghrelin_acylated_pp[['id_participant', 'session']])

# Find outliers based on Ghrelin_acylated zscore per session
outliers_Ghrelin_acylated_session_pp = df_GI_bid_hi_pp_Ghrelin_acylated[(df_GI_bid_hi_pp_Ghrelin_acylated['Ghrelin_acylated_zscore_session'] > 3) | (df_GI_bid_hi_pp_Ghrelin_acylated['Ghrelin_acylated_zscore_session'] < -3)]
print(outliers_Ghrelin_acylated_session_pp[['id_participant', 'session']])

# Remove outliers based on z-score threshold of 3
df_GI_bid_hi_pp_Ghrelin_acylated_no_outliers = df_GI_bid_hi_pp_Ghrelin_acylated[df_GI_bid_hi_pp_Ghrelin_acylated['Ghrelin_acylated_zscore'].abs() <= 3]
#print(basic_stat(df_GI_bid_hi_pp_Ghrelin_acylated_no_outliers))

# Remove outliers based on z-score threshold of 3 for session
df_GI_bid_hi_pp_Ghrelin_acylated_no_outliers_session = df_GI_bid_hi_pp_Ghrelin_acylated[df_GI_bid_hi_pp_Ghrelin_acylated['Ghrelin_acylated_zscore_session'].abs() <= 3]
#print(basic_stat(df_GI_bid_hi_pp_Ghrelin_acylated_no_outliers_session))

### Create DFs for PYY PP

In [ ]:
# Find participants and sessions where PYY is not recorded (PP)
df_no_PYY_pp = df_GI_bid_hi_pp[df_GI_bid_hi_pp['PYY'].isna()][['id_participant', 'session']].drop_duplicates()
print(df_no_PYY_pp)

# Create a DF where there are no Nans for PYY PP
df_GI_bid_hi_pp_PYY = df_GI_bid_hi_pp.dropna(subset=['PYY'])

# Calculate z-scores for PYY and session
df_GI_bid_hi_pp_PYY['PYY_zscore_session'] = df_GI_bid_hi_pp_PYY.groupby('session')['PYY'].transform(lambda x: (x - x.mean()) / x.std())
df_GI_bid_hi_pp_PYY['PYY_zscore'] = zscore(df_GI_bid_hi_pp_PYY['PYY'])
#print(df_GI_bid_hi_pp_PYY['PYY_zscore_session'])

# Print PYY_zscore_session and PYY_zscore
#print(df_GI_bid_hi_pp_PYY[['PYY_zscore_session', 'PYY_zscore']])

#print(basic_stat(df_GI_bid_hi_pp_PYY))

# Find outliers based on PYY zscore
outliers_PYY_pp = df_GI_bid_hi_pp_PYY[(df_GI_bid_hi_pp_PYY['PYY_zscore'] > 3) | (df_GI_bid_hi_pp_PYY['PYY_zscore'] < -3)]
print(outliers_PYY_pp[['id_participant', 'session']])

# Find outliers based on PYY zscore per session
outliers_PYY_session_pp = df_GI_bid_hi_pp_PYY[(df_GI_bid_hi_pp_PYY['PYY_zscore_session'] > 3) | (df_GI_bid_hi_pp_PYY['PYY_zscore_session'] < -3)]
print(outliers_PYY_session_pp[['id_participant', 'session']])

# Remove outliers based on z-score threshold of 3
df_GI_bid_hi_pp_PYY_no_outliers = df_GI_bid_hi_pp_PYY[df_GI_bid_hi_pp_PYY['PYY_zscore'].abs() <= 3]
#print(basic_stat(df_GI_bid_hi_pp_PYY_no_outliers))

# Remove outliers based on z-score threshold of 3 for session
df_GI_bid_hi_pp_PYY_no_outliers_session = df_GI_bid_hi_pp_PYY[df_GI_bid_hi_pp_PYY['PYY_zscore_session'].abs() <= 3]
#print(basic_stat(df_GI_bid_hi_pp_PYY_no_outliers_session))

### Create Dfs Ghreline Totale PP

In [ ]:
# Find participants and sessions where Ghrelin_Totale is not recorded (PP)
df_no_Ghrelin_Totale_pp = df_GI_bid_hi_pp[df_GI_bid_hi_pp['Ghrelin_Totale'].isna()][['id_participant', 'session']].drop_duplicates()
print(df_no_Ghrelin_Totale_pp)

# Create a DF where there are no Nans for Ghrelin_Totale PP
df_GI_bid_hi_pp_Ghrelin_Totale = df_GI_bid_hi_pp.dropna(subset=['Ghrelin_Totale'])

# Calculate z-scores for Ghrelin_Totale and session
df_GI_bid_hi_pp_Ghrelin_Totale['Ghrelin_Totale_zscore_session'] = df_GI_bid_hi_pp_Ghrelin_Totale.groupby('session')['Ghrelin_Totale'].transform(lambda x: (x - x.mean()) / x.std())
df_GI_bid_hi_pp_Ghrelin_Totale['Ghrelin_Totale_zscore'] = zscore(df_GI_bid_hi_pp_Ghrelin_Totale['Ghrelin_Totale'])
#print(df_GI_bid_hi_pp_Ghrelin_Totale['Ghrelin_Totale_zscore_session'])

# Print Ghrelin_Totale_zscore_session and Ghrelin_Totale_zscore
#print(df_GI_bid_hi_pp_Ghrelin_Totale[['Ghrelin_Totale_zscore_session', 'Ghrelin_Totale_zscore']])

#print(basic_stat(df_GI_bid_hi_pp_Ghrelin_Totale))

# Find outliers based on Ghrelin_Totale zscore
outliers_Ghrelin_Totale_pp = df_GI_bid_hi_pp_Ghrelin_Totale[(df_GI_bid_hi_pp_Ghrelin_Totale['Ghrelin_Totale_zscore'] > 3) | (df_GI_bid_hi_pp_Ghrelin_Totale['Ghrelin_Totale_zscore'] < -3)]
print(outliers_Ghrelin_Totale_pp[['id_participant', 'session']])

# Find outliers based on Ghrelin_Totale zscore per session
outliers_Ghrelin_Totale_session_pp = df_GI_bid_hi_pp_Ghrelin_Totale[(df_GI_bid_hi_pp_Ghrelin_Totale['Ghrelin_Totale_zscore_session'] > 3) | (df_GI_bid_hi_pp_Ghrelin_Totale['Ghrelin_Totale_zscore_session'] < -3)]
print(outliers_Ghrelin_Totale_session_pp[['id_participant', 'session']])

# Remove outliers based on z-score threshold of 3
df_GI_bid_hi_pp_Ghrelin_Totale_no_outliers = df_GI_bid_hi_pp_Ghrelin_Totale[df_GI_bid_hi_pp_Ghrelin_Totale['Ghrelin_Totale_zscore'].abs() <= 3]
#print(basic_stat(df_GI_bid_hi_pp_Ghrelin_Totale_no_outliers))

# Remove outliers based on z-score threshold of 3 for session
df_GI_bid_hi_pp_Ghrelin_Totale_no_outliers_session = df_GI_bid_hi_pp_Ghrelin_Totale[df_GI_bid_hi_pp_Ghrelin_Totale['Ghrelin_Totale_zscore_session'].abs() <= 3]
#print(basic_stat(df_GI_bid_hi_pp_Ghrelin_Totale_no_outliers_session))

## Create df_GI_bid_hi_pp_no_outlier

In [ ]:
df_GI_bid_hi_pp_no_outlier = df_GI_bid_no_outlier.copy()
df_GI_bid_hi_pp_no_outlier = df_GI_bid_hi_pp_no_outlier[(df_GI_bid_hi_pp_no_outlier['fed_state'] == 'pp') & (df_GI_bid_hi_pp['calories'] == 'hi')]

# Exploratory Plots with outliers

## AC, Outliers included

In [ ]:
# Assuming df_GI_bid_hi_pp is already defined
columns_to_plot = ['bid','GLP_1_total', 'Leptin', 'PYY', 'Ghrelin_Unacylated', 'Ghrelin_acylated','Ghrelin_Totale']

# Calculate the average per session for each column
average_per_session = df_GI_bid_hi_ac.groupby('session')[columns_to_plot].mean()

# Create a matrix of subplots
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(15, 10))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plotting
for i, column in enumerate(columns_to_plot):
    axes[i].plot(average_per_session.index, average_per_session[column], marker='o')
    axes[i].set_title(f'Average {column} per Session')
    axes[i].set_xlabel('Session')
    axes[i].set_ylabel(column)
    axes[i].grid(True)

# Remove the empty subplot (if any)
if len(columns_to_plot) < len(axes):
    fig.delaxes(axes[-1])

plt.tight_layout()
plt.show()

## PP, line plots, Outliers included

In [ ]:
# Assuming df_GI_bid_hi_pp is already defined
columns_to_plot = ['bid','GLP_1_total', 'Leptin', 'PYY', 'Ghrelin_Unacylated', 'Ghrelin_acylated','Ghrelin_Totale']

# Calculate the average per session for each column
average_per_session = df_GI_bid_hi_pp.groupby('session')[columns_to_plot].mean()

# Create a matrix of subplots
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(15, 10))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plotting
for i, column in enumerate(columns_to_plot):
    axes[i].plot(average_per_session.index, average_per_session[column], marker='o')
    axes[i].set_title(f'Average {column} per Session')
    axes[i].set_xlabel('Session')
    axes[i].set_ylabel(column)
    axes[i].grid(True)

# Remove the empty subplot (if any)
if len(columns_to_plot) < len(axes):
    fig.delaxes(axes[-1])

plt.tight_layout()
plt.show()

## AC, Scatterplot, outliers included

In [ ]:

# Assuming df_GI_bid_hi_pp is already defined
columns_to_plot = ['GLP_1_total', 'Leptin', 'PYY', 'Ghrelin_Unacylated', 'Ghrelin_acylated','Ghrelin_Totale']
sessions = df_GI_bid_hi_ac['session'].unique()

# Determine the grid size
num_sessions = len(sessions)
num_columns = len(columns_to_plot)
fig, axes = plt.subplots(num_sessions, num_columns, figsize=(15, 10))

# Plotting scatter plots for each session and each column
for i, session in enumerate(sessions):
    session_data = df_GI_bid_hi_ac[df_GI_bid_hi_ac['session'] == session]
    for j, column in enumerate(columns_to_plot):
        ax = axes[i, j]
        ax.set_xlim(0, 5)
        ax.scatter(session_data['bid'], session_data[column], label=f'Session {session}')
        ax.set_title(f'{column} (Session {session})')
        ax.set_xlabel('Bid')
        ax.set_ylabel(column)
        ax.grid(True)

# Adjust layout
plt.tight_layout()
plt.show()

## PP. scatterplot, outliers included

In [ ]:

# Assuming df_GI_bid_hi_pp is already defined
columns_to_plot = ['GLP_1_total', 'Leptin', 'PYY', 'Ghrelin_Unacylated', 'Ghrelin_acylated','Ghrelin_Totale']
sessions = df_GI_bid_hi_pp['session'].unique()

# Determine the grid size
num_sessions = len(sessions)
num_columns = len(columns_to_plot)
fig, axes = plt.subplots(num_sessions, num_columns, figsize=(15, 10))

# Plotting scatter plots for each session and each column
for i, session in enumerate(sessions):
    session_data = df_GI_bid_hi_pp[df_GI_bid_hi_pp['session'] == session]
    for j, column in enumerate(columns_to_plot):
        ax = axes[i, j]
        ax.scatter(session_data['bid'], session_data[column], label=f'Session {session}')
        ax.set_title(f'{column} (Session {session})')
        ax.set_xlabel('Bid')
        ax.set_ylabel(column)
        ax.grid(True)

# Adjust layout
plt.tight_layout()
plt.show()

# Exploratory plots no outlier

### AC , one DF for all plots (less accurate analysis here)

In [ ]:
# Assuming df_GI_bid_hi_pp is already defined
columns_to_plot = ['bid','GLP_1_total', 'Leptin', 'PYY', 'Ghrelin_Unacylated', 'Ghrelin_acylated','Ghrelin_Totale']

# Calculate the average per session for each column
average_per_session = df_GI_bid_hi_ac_no_outlier.groupby('session')[columns_to_plot].mean()

# Create a matrix of subplots
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(15, 10))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plotting
for i, column in enumerate(columns_to_plot):
    axes[i].plot(average_per_session.index, average_per_session[column], marker='o')
    axes[i].set_title(f'Average {column} per Session')
    axes[i].set_xlabel('Session')
    axes[i].set_ylabel(column)
    axes[i].grid(True)

# Remove the empty subplot (if any)
if len(columns_to_plot) < len(axes):
    fig.delaxes(axes[-1])

plt.tight_layout()
plt.show()

### AC with dfs per hormone

In [ ]:
# Define the DataFrames for each column
df_dict = {
    'bid': df_GI_bid_hi_ac,
    'GLP_1_total': df_GI_bid_hi_ac_GLP1_no_outliers_session,
    'Leptin': df_GI_bid_hi_ac_Leptin_no_outliers_session,
    'PYY': df_GI_bid_hi_ac_PYY_no_outliers_session,
    'Ghrelin_Unacylated': df_GI_bid_hi_ac_Ghrelin_Unacylated_no_outliers_session,
    'Ghrelin_acylated': df_GI_bid_hi_ac_Ghrelin_acylated_no_outliers_session,
    'Ghrelin_Totale': df_GI_bid_hi_ac_Ghrelin_Totale_no_outliers_session
}

# Define the columns to plot
columns_to_plot = ['bid', 'GLP_1_total', 'Leptin', 'PYY', 'Ghrelin_Unacylated', 'Ghrelin_acylated', 'Ghrelin_Totale']

# Create a matrix of subplots
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(15, 10))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plotting
for i, column in enumerate(columns_to_plot):
    df = df_dict[column]
    # Calculate the average per session for each column
    average_per_session = df.groupby('session')[column].mean()
    
    axes[i].plot(average_per_session.index, average_per_session, marker='o')
    axes[i].set_title(f'Average {column} per Session')
    axes[i].set_xlabel('Session')
    axes[i].set_ylabel(column)
    axes[i].grid(True)

# Remove the empty subplot (if any)
if len(columns_to_plot) < len(axes):
    fig.delaxes(axes[-1])

plt.tight_layout()
plt.show()

### Scatter plot one DF for all hormones, no outliers, AC (less accurate analysis)

In [ ]:
# Assuming df_GI_bid_hi_pp is already defined
columns_to_plot = ['GLP_1_total', 'Leptin', 'PYY', 'Ghrelin_Unacylated', 'Ghrelin_acylated','Ghrelin_Totale']
sessions = df_GI_bid_hi_ac_no_outlier['session'].unique()

# Determine the grid size
num_sessions = len(sessions)
num_columns = len(columns_to_plot)
fig, axes = plt.subplots(num_sessions, num_columns, figsize=(15, 10))

# Plotting scatter plots for each session and each column
for i, session in enumerate(sessions):
    session_data = df_GI_bid_hi_ac_no_outlier[df_GI_bid_hi_ac_no_outlier['session'] == session]
    for j, column in enumerate(columns_to_plot):
        ax = axes[i, j]
        ax.scatter(session_data['bid'], session_data[column], label=f'Session {session}')
        ax.set_title(f'{column} (Session {session})')
        ax.set_xlabel('Bid')
        ax.set_ylabel(column)
        ax.grid(True)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Assuming df_GI_bid_hi_pp is already defined
columns_to_plot = ['GLP_1_total', 'Leptin', 'PYY', 'Ghrelin_Unacylated', 'Ghrelin_acylated','Ghrelin_Totale']

# Determine the grid size
num_columns = len(columns_to_plot)
fig, axes = plt.subplots(1, num_columns, figsize=(20, 5))

# Plotting scatter plots for each column with all sessions combined
for i, column in enumerate(columns_to_plot):
    ax = axes[i]
    sns.scatterplot(data=df_GI_bid_hi_ac_no_outlier, x='bid', y=column, hue='session', ax=ax)
    
    # Fit regression line
    slope, intercept, r_value, p_value, std_err = linregress(df_GI_bid_hi_ac_no_outlier['bid'], df_GI_bid_hi_ac_no_outlier[column])
    ax.plot(df_GI_bid_hi_ac_no_outlier['bid'], intercept + slope * df_GI_bid_hi_ac_no_outlier['bid'], color='red')
    
    # Add R² value to the plot
    ax.text(0.05, 0.95, f'$R^2 = {r_value**2:.2f}$', transform=ax.transAxes, fontsize=12, verticalalignment='top')
    
    ax.set_title(f'{column}')
    ax.set_xlabel('Bid')
    ax.set_ylabel(column)
    ax.grid(True)

# Adjust layout
plt.tight_layout()
plt.show()

### Scatterplot, One df per hormone, no outliers AC, better analysis

In [ ]:

# Define the DataFrames for each column
df_dict = {
    'GLP_1_total': df_GI_bid_hi_ac_GLP1_no_outliers_session,
    'Leptin': df_GI_bid_hi_ac_Leptin_no_outliers_session,
    'PYY': df_GI_bid_hi_ac_PYY_no_outliers_session,
    'Ghrelin_Unacylated': df_GI_bid_hi_ac_Ghrelin_Unacylated_no_outliers_session,
    'Ghrelin_acylated': df_GI_bid_hi_ac_Ghrelin_acylated_no_outliers_session,
    'Ghrelin_Totale': df_GI_bid_hi_ac_Ghrelin_Totale_no_outliers_session
}

# Define the columns to plot
columns_to_plot = ['GLP_1_total', 'Leptin', 'PYY', 'Ghrelin_Unacylated', 'Ghrelin_acylated', 'Ghrelin_Totale']

# Assuming df_GI_bid_hi_ac is already defined
sessions = df_GI_bid_hi_ac['session'].unique()

# Check if there are any sessions available
if len(sessions) > 0:
    # Determine the grid size
    num_sessions = len(sessions)
    num_columns = len(columns_to_plot)
    fig, axes = plt.subplots(num_sessions, num_columns, figsize=(15, 10))

    # Plotting scatter plots for each session and each column
    for i, session in enumerate(sessions):
        for j, column in enumerate(columns_to_plot):
            df = df_dict[column]
            session_data = df[df['session'] == session]
            bid_data = df_GI_bid_hi_ac[df_GI_bid_hi_ac['session'] == session]['bid']
            # Ensure bid_data and session_data[column] have the same length
            common_index = session_data.index.intersection(bid_data.index)
            ax = axes[i, j]
            ax.scatter(bid_data.loc[common_index], session_data.loc[common_index, column], label=f'Session {session}')
            ax.set_title(f'{column} (Session {session})')
            ax.set_xlabel('Bid')
            ax.set_ylabel(column)
            ax.grid(True)

    # Adjust layout
    plt.tight_layout()
    plt.show()
else:
    print("No sessions available to plot.")

### PP, one df for all hormones, not most accurate, no outliers

In [ ]:
# Assuming df_GI_bid_hi_pp is already defined
columns_to_plot = ['bid','GLP_1_total', 'Leptin', 'PYY', 'Ghrelin_Unacylated', 'Ghrelin_acylated', 'Ghrelin_Totale']

# Calculate the average per session for each column
average_per_session = df_GI_bid_hi_pp_no_outlier.groupby('session')[columns_to_plot].mean()

# Create a matrix of subplots
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(15, 10))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plotting
for i, column in enumerate(columns_to_plot):
    axes[i].plot(average_per_session.index, average_per_session[column], marker='o')
    axes[i].set_title(f'Average {column} per Session')
    axes[i].set_xlabel('Session')
    axes[i].set_ylabel(column)
    axes[i].grid(True)

# Remove the empty subplot (if any)
if len(columns_to_plot) < len(axes):
    fig.delaxes(axes[-1])

plt.tight_layout()
plt.show()

### PP, no outliers, one DF per hormone (most accurate here)

In [ ]:
# Define the DataFrames for each column
df_dict = {
    'bid': df_GI_bid_hi_ac,
    'GLP_1_total': df_GI_bid_hi_pp_GLP1_no_outliers_session,
    'Leptin': df_GI_bid_hi_pp_Leptin_no_outliers_session,
    'PYY': df_GI_bid_hi_pp_PYY_no_outliers_session,
    'Ghrelin_Unacylated': df_GI_bid_hi_pp_Ghrelin_Unacylated_no_outliers_session,
    'Ghrelin_acylated': df_GI_bid_hi_pp_Ghrelin_acylated_no_outliers_session,
    'Ghrelin_Totale': df_GI_bid_hi_pp_Ghrelin_Totale_no_outliers_session
}

# Define the columns to plot
columns_to_plot = ['bid', 'GLP_1_total', 'Leptin', 'PYY', 'Ghrelin_Unacylated', 'Ghrelin_acylated', 'Ghrelin_Totale']

# Create a matrix of subplots
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(15, 10))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plotting
for i, column in enumerate(columns_to_plot):
    df = df_dict[column]
    # Calculate the average per session for each column
    average_per_session = df.groupby('session')[column].mean()
    
    axes[i].plot(average_per_session.index, average_per_session, marker='o')
    axes[i].set_title(f'Average {column} per Session')
    axes[i].set_xlabel('Session')
    axes[i].set_ylabel(column)
    axes[i].grid(True)

# Remove the empty subplot (if any)
if len(columns_to_plot) < len(axes):
    fig.delaxes(axes[-1])

plt.tight_layout()
plt.show()

### Scatterplot, one df for all (less accurate), PP, no outliers

In [ ]:
# Assuming df_GI_bid_hi_pp is already defined
columns_to_plot = ['GLP_1_total', 'Leptin', 'PYY', 'Ghrelin_Unacylated', 'Ghrelin_acylated','Ghrelin_Totale']
sessions = df_GI_bid_hi_pp_no_outlier['session'].unique()

# Determine the grid size
num_sessions = len(sessions)
num_columns = len(columns_to_plot)
fig, axes = plt.subplots(num_sessions, num_columns, figsize=(15, 10))

# Plotting scatter plots for each session and each column
for i, session in enumerate(sessions):
    session_data = df_GI_bid_hi_pp_no_outlier[df_GI_bid_hi_pp_no_outlier['session'] == session]
    for j, column in enumerate(columns_to_plot):
        ax = axes[i, j]
        ax.scatter(session_data['bid'], session_data[column], label=f'Session {session}')
        ax.set_title(f'{column} (Session {session})')
        ax.set_xlabel('Bid')
        ax.set_ylabel(column)
        ax.grid(True)

# Adjust layout
plt.tight_layout()
plt.show()

### Scatterplot, one df per hormone (most accurate), no outliers, PP

In [ ]:
# Define the DataFrames for each column
df_dict = {
    'GLP_1_total': df_GI_bid_hi_pp_GLP1_no_outliers_session,
    'Leptin': df_GI_bid_hi_pp_Leptin_no_outliers_session,
    'PYY': df_GI_bid_hi_pp_PYY_no_outliers_session,
    'Ghrelin_Unacylated': df_GI_bid_hi_pp_Ghrelin_Unacylated_no_outliers_session,
    'Ghrelin_acylated': df_GI_bid_hi_pp_Ghrelin_acylated_no_outliers_session,
    'Ghrelin_Totale': df_GI_bid_hi_pp_Ghrelin_Totale_no_outliers_session
}

# Define the columns to plot
columns_to_plot = ['GLP_1_total', 'Leptin', 'PYY', 'Ghrelin_Unacylated', 'Ghrelin_acylated', 'Ghrelin_Totale']

# Assuming df_GI_bid_hi_pp is already defined
sessions = df_GI_bid_hi_pp['session'].unique()

# Check if there are any sessions available
if len(sessions) > 0:
    # Determine the grid size
    num_sessions = len(sessions)
    num_columns = len(columns_to_plot)
    fig, axes = plt.subplots(num_sessions, num_columns, figsize=(15, 10))

    # Plotting scatter plots for each session and each column
    for i, session in enumerate(sessions):
        for j, column in enumerate(columns_to_plot):
            df = df_dict[column]
            session_data = df[df['session'] == session]
            bid_data = df_GI_bid_hi_pp[df_GI_bid_hi_pp['session'] == session]['bid']
            # Ensure bid_data and session_data[column] have the same length
            common_index = session_data.index.intersection(bid_data.index)
            ax = axes[i, j]
            ax.scatter(bid_data.loc[common_index], session_data.loc[common_index, column], label=f'Session {session}')
            ax.set_title(f'{column} (Session {session})')
            ax.set_xlabel('Bid')
            ax.set_ylabel(column)
            ax.grid(True)

    # Adjust layout
    plt.tight_layout()
    plt.show()
else:
    print("No sessions available to plot.")

# Type Chx vs bid high

### bid high vs type chx

In [ ]:
# Assuming df_GI_bid_hi_pp_no_outlier is already defined
plt.figure(figsize=(10, 6))

# Plotting the data
sns.lineplot(data=bid_avg_hi, x='session', y='bid', hue='type_chx', marker='o')

# Adding title and labels
plt.xlabel('Session')
plt.ylabel('Mises ($)')
plt.legend(title='Type de chirurgie')
plt.ylim(0, 5)

# Relabeling the legend
legend_labels = {1: 'Gastrectomie', 2: 'ROUX-en-Y', 3: 'DYBP-SG'}
handles, labels = plt.gca().get_legend_handles_labels()
new_labels = [legend_labels[int(float(label))] for label in labels]  # Convert to float first, then to int
plt.legend(handles, new_labels, title='Surgery Type')

# Set custom x-axis labels
plt.xticks(ticks=[0, 1, 2, 3], labels=['pré-chirurgie', '4 mois', '12 mois', '24 mois'])

# Display the plot
plt.show()

In [ ]:
# Define the formula
formula = 'bid ~ type_chx + age  + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, bid_avg_hi, groups=bid_avg_hi['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")



### Contrast type roux y vs dybp

In [ ]:
list_of_values = [0,-1,1,0,0,0]  # Corrected list with 12 elements
contrast_matrix = np.array([list_of_values])  # Convert to 2D array
print(contrast_matrix.shape)  # Should print (1, 12)

# Perform the contrasts
contrast_results = mixed_model_fit.t_test(contrast_matrix)

# Print the contrast results
print(contrast_results)

### Contrast dybp vs gastrectomie

In [ ]:
list_of_values = [0,0,1,0,0,0]  # Corrected list with 12 elements
contrast_matrix = np.array([list_of_values])  # Convert to 2D array
print(contrast_matrix.shape)  # Should print (1, 12)

# Perform the contrasts
contrast_results = mixed_model_fit.t_test(contrast_matrix)

# Print the contrast results
print(contrast_results)

### Contraste roux-y vs gastrectomie

In [ ]:
list_of_values = [0,1,0,0,0,0]  # Corrected list with 12 elements
contrast_matrix = np.array([list_of_values])  # Convert to 2D array
print(contrast_matrix.shape)  # Should print (1, 12)

# Perform the contrasts
contrast_results = mixed_model_fit.t_test(contrast_matrix)

# Print the contrast results
print(contrast_results)

### bid = type_chx * session

In [ ]:
# Define the formula
formula = 'bid ~ type_chx*session + age  + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, bid_avg_hi, groups=bid_avg_hi['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")



### Contraste type chx * session

In [ ]:
list_of_values = [0,0,0,0,0,0,0,0,0,1,-1,0,0,0,0]  # Corrected list with 12 elements
contrast_matrix = np.array([list_of_values])  # Convert to 2D array
print(contrast_matrix.shape)  # Should print (1, 12)

# Perform the contrasts
contrast_results = mixed_model_fit.t_test(contrast_matrix)

# Print the contrast results
print(contrast_results)

# Mixed models with outliers

## mixed model Post prandial

### GLP-1 

In [ ]:
# Define the formula
formula = 'bid ~ GLP_1_total + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_GLP1, groups=df_GI_bid_hi_pp_GLP1['id_participant'], re_formula='1')
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

# Perform a Wald test
print("Wald test results:")
w_matrix = np.eye(len(mixed_model_fit.params))  # Identity matrix for testing all fixed effects
wald_test = mixed_model_fit.wald_test_terms()

# Print the Wald test results with more precision for p-values
pd.set_option('display.float_format', lambda x: '%.10f' % x)
print(wald_test)

print(f"AIC: {aic}")
print(f"BIC: {bic}")

# Calculate R-squared for each parameter
r_squared_data = pd.DataFrame(columns=["Parameter", "R_squared"])

for param in mixed_model_fit.model.exog_names:
    # Create a model with only the current parameter
    param = param.split('[')[0]

    formula_single = f'bid ~ {param}'
    if param == 'Intercept':
        continue
    single_model = smf.mixedlm(formula_single, df_GI_bid_hi_pp_GLP1, groups=df_GI_bid_hi_pp_GLP1['id_participant'], re_formula='1')
    single_model_fit = single_model.fit()
    
    # Calculate the R-squared as the proportion of variance explained
    total_variance = np.var(df_GI_bid_hi_pp_GLP1['bid'])
    explained_variance = total_variance - np.var(single_model_fit.resid)
    r_squared = explained_variance / total_variance
    
    # Append the result
    r_squared_data = pd.concat([r_squared_data, pd.DataFrame([{"Parameter": param, "R_squared": r_squared}])], ignore_index=True)

# Convert R-squared to percentage
r_squared_data["R_squared"] = r_squared_data["R_squared"] * 100

# Print the R-squared values
print(r_squared_data)

### Graph propre GLP-1

In [ ]:
# Calculate the average GLP_1_total per session
average_glp1 = df_GI_bid_hi_pp_GLP1.groupby('session')['GLP_1_total'].mean()

# Calculate the average bid per session
average_bid = df_GI_bid_hi_pp_GLP1.groupby('session')['bid'].mean()

# Plotting average GLP_1_total per session with labels
fig, ax1 = plt.subplots(figsize=(10, 6))

# Primary y-axis for GLP_1_total
ax1.plot(average_glp1.index, average_glp1.values, marker='o', linestyle='-', color='b', label='GLP-1 Total')
ax1.set_xlabel('Session')
ax1.set_ylabel('GLP-1 Total (pmol/l)', color='black')
ax1.tick_params(axis='y', labelcolor='black')
ax1.grid(True)
ax1.set_ylim(0, 450)  # Set y-axis limits for GLP-1 Total

# Ensure y-axis ticks are integers
ax1.yaxis.set_major_locator(plt.MaxNLocator(integer=True))

# Set custom x-axis labels
ax1.set_xticks(average_glp1.index)
ax1.set_xticklabels(['pré-chirurgie', '4 mois', '12 mois', '24 mois'])

# Secondary y-axis for bid
ax2 = ax1.twinx()
ax2.plot(average_bid.index, average_bid.values, marker='o', linestyle='-', color='orange', label='Mises pour aliments riches en calories')
ax2.set_ylabel('Mises ($)', color='black')
ax2.tick_params(axis='y', labelcolor='black')
ax2.set_ylim(0, 5)  # Set y-axis limits for bid
# Calculate the standard deviation of bid per session
std_bid = df_GI_bid_hi_pp_GLP1.groupby('session')['bid'].std()

# Adding labels for average_bid
for i, (avg, std) in enumerate(zip(average_bid.values, std_bid.values)):
    x_coord = i  # Use the index position directly for alignment
    if i == 0:  # Session 1
        ax2.text(x_coord + 0.2, avg + 0.3, f'{avg:.2f} ± {std:.2f}', ha='center', va='top', color='orange')  # Above and centered
    else:  # Sessions 2, 3, 4
        ax2.text(x_coord - 0.1, avg + 0.4, f'{avg:.2f} ± {std:.2f}', ha='center', va='top', color='orange')  # Below and centered

# Calculate the standard deviation of GLP_1_total per session
std_glp1 = df_GI_bid_hi_pp_GLP1.groupby('session')['GLP_1_total'].std()

# Adding labels for average_glp1
for i, (avg, std) in enumerate(zip(average_glp1.values, std_glp1.values)):
    x_coord = i  # Use the index position directly for alignment
    if i == 0:  # Session 1
        ax1.text(x_coord + 0.2, avg - 50, f'{avg:.2f} ± {std:.2f}', ha='center', va='top', color='blue')  # Above and centered
    else:  # Sessions 2, 3, 4
        ax1.text(x_coord - 0.1, avg - 50, f'{avg:.2f} ± {std:.2f}', ha='center', va='bottom', color='blue')  # Below and centered

# Adding legends
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

ax1.grid(False)
plt.grid(False)



plt.show()

### GLP1 interaction chx

In [ ]:
# Define the formula
formula = 'bid ~ GLP_1_total*type_chx + age  + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_GLP1, groups=df_GI_bid_hi_pp_GLP1['id_participant'], re_formula='1')
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")

### GLP1 session 1 

In [ ]:


# Define the formula
formula = 'bid ~ GLP_1_total + type_chx + age  + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_GLP1_ses_1, groups=df_GI_bid_hi_pp_GLP1_ses_1['id_participant'], re_formula='1')
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")

### GLP1 session 2

In [ ]:


# Define the formula
formula = 'bid ~ GLP_1_total + type_chx + age  + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_GLP1_ses_2, groups=df_GI_bid_hi_pp_GLP1_ses_2['id_participant'], re_formula='1')
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")

### GLP1-Session 3

In [ ]:


# Define the formula
formula = 'bid ~ GLP_1_total + type_chx + age  + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_GLP1_ses_3, groups=df_GI_bid_hi_pp_GLP1_ses_3['id_participant'], re_formula='1')
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")

### GLP1 session 4

In [ ]:


# Define the formula
formula = 'bid ~ GLP_1_total + type_chx + age  + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_GLP1_ses_4, groups=df_GI_bid_hi_pp_GLP1_ses_4['id_participant'], re_formula='1')
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")

### Ghrelin_Unacylated

In [ ]:
# Define the formula
formula = 'bid ~ Ghrelin_Unacylated + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_Ghrelin_Unacylated, groups=df_GI_bid_hi_pp_Ghrelin_Unacylated['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")



### Ghrelin unacetylated interaction chx

In [ ]:
# Define the formula
formula = 'bid ~ Ghrelin_Unacylated*type_chx + age  + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_Ghrelin_Unacylated, groups=df_GI_bid_hi_pp_Ghrelin_Unacylated['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")



### Ghrelin_acylated

In [ ]:
# Define the formula
formula = 'bid ~ Ghrelin_acylated + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_Ghrelin_acylated, groups=df_GI_bid_hi_pp_Ghrelin_acylated['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


### Ghrelyin acylated interaction chx

In [ ]:
# Define the formula
formula = 'bid ~ Ghrelin_acylated*type_chx + age + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_Ghrelin_acylated, groups=df_GI_bid_hi_pp_Ghrelin_acylated['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


### Leptin

In [ ]:
# Define the formula
formula = 'bid ~ Leptin + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_Leptin, groups=df_GI_bid_hi_pp_Leptin['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()



# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")

### PYY

In [ ]:
basic_stat(df_GI_bid_hi_pp_PYY)

In [ ]:
# Define the formula
formula = 'bid ~ PYY + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_PYY, groups=df_GI_bid_hi_pp_PYY['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Announce that we are printing the Wald test results
print("Wald test results:")

# Perform a Wald test
w_matrix = np.eye(len(mixed_model_fit.params))  # Identity matrix for testing all fixed effects
wald_test = mixed_model_fit.wald_test_terms()

# Print the Wald test results with more precision for p-values
pd.set_option('display.float_format', lambda x: '%.10f' % x)
print(wald_test)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

# Perform a Wald test
print("Wald test results:")
w_matrix = np.eye(len(mixed_model_fit.params))  # Identity matrix for testing all fixed effects
wald_test = mixed_model_fit.wald_test_terms()

# Print the Wald test results with more precision for p-values
pd.set_option('display.float_format', lambda x: '%.10f' % x)
print(wald_test)

print(f"AIC: {aic}")
print(f"BIC: {bic}")


In [ ]:


max_pyy_value = df_GI_bid_hi_pp_PYY['PYY'].max()
max_pyy_value


### Graph au propre pyy

In [ ]:
# Calculate the average PYY per session
average_pyy = df_GI_bid_hi_pp_PYY.groupby('session')['PYY'].mean()

# Calculate the average bid per session
average_bid = df_GI_bid_hi_pp_PYY.groupby('session')['bid'].mean()

# Plotting average PYY per session with labels
fig, ax1 = plt.subplots(figsize=(10, 6))

# Primary y-axis for PYY
ax1.plot(average_pyy.index, average_pyy.values, marker='o', linestyle='-', color='b', label='PYY')
ax1.set_xlabel('Session')
ax1.set_ylabel('PYY (pmol/l)', color='black')
ax1.tick_params(axis='y', labelcolor='black')
ax1.grid(True)
ax1.set_ylim(0, 300)


# Ensure y-axis ticks are integers
ax1.yaxis.set_major_locator(plt.MaxNLocator(integer=True))

# Set custom x-axis labels
ax1.set_xticks(average_pyy.index)
ax1.set_xticklabels(['pré-chirurgie', '4 mois', '12 mois', '24 mois'])

# Secondary y-axis for bid
ax2 = ax1.twinx()
ax2.plot(average_bid.index, average_bid.values, marker='o', linestyle='-', color='orange', label='Mises pour aliments riches en calories')
ax2.set_ylabel('Mises ($)', color='black')
ax2.tick_params(axis='y', labelcolor='black')
ax2.set_ylim(0, 5)

# Calculate the standard deviation of bid per session
std_bid = df_GI_bid_hi_pp_PYY.groupby('session')['bid'].std()

# Adding labels for average_bid
for i, (avg, std) in enumerate(zip(average_bid.values, std_bid.values)):
    x_coord = i  # Use the index position directly for alignment
    if i == 0:  # Session 1
        ax2.text(x_coord + 0.1, avg + 0.3, f'{avg:.2f} ± {std:.2f}', ha='center', va='top', color='orange')  # Above and centered
    else:  # Sessions 2, 3, 4
        ax2.text(x_coord - 0.1, avg + 0.4, f'{avg:.2f} ± {std:.2f}', ha='center', va='top', color='orange')  # Below and centered

# Calculate the standard deviation of PYY per session
std_pyy = df_GI_bid_hi_pp_PYY.groupby('session')['PYY'].std()

# Adding labels for average_pyy
for i, (avg, std) in enumerate(zip(average_pyy.values, std_pyy.values)):
    x_coord = i  # Use the index position directly for alignment
    if i == 0:  # Session 1
        ax1.text(x_coord + 0.1, avg - 10, f'{avg:.2f} ± {std:.2f}', ha='center', va='top', color='blue')  # Above and centered
    else:  # Sessions 2, 3, 4
        ax1.text(x_coord - 0.1, avg - 30, f'{avg:.2f} ± {std:.2f}', ha='center', va='bottom', color='blue')  # Below and centered

# Adding legends
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

ax1.grid(False)
plt.grid(False)

plt.show()

### PYY and interaction chx

In [ ]:
# Define the formula
formula = 'bid ~ PYY*type_chx + age + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_PYY, groups=df_GI_bid_hi_pp_PYY['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Announce that we are printing the Wald test results
print("Wald test results:")

# Perform a Wald test
w_matrix = np.eye(len(mixed_model_fit.params))  # Identity matrix for testing all fixed effects
wald_test = mixed_model_fit.wald_test_terms()

# Print the Wald test results with more precision for p-values
pd.set_option('display.float_format', lambda x: '%.10f' % x)
print(wald_test)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


### Ghreline Totale

In [ ]:
# Define the formula
formula = 'bid ~ Ghrelin_Totale + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_Ghrelin_Totale, groups=df_GI_bid_hi_pp_Ghrelin_Totale['id_participant'])
mixed_model_fit = mixed_model.fit()



# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


### Ghreline totale interaction chx

In [ ]:
# Define the formula
formula = 'bid ~ Ghrelin_Totale*type_chx + age  + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_Ghrelin_Totale, groups=df_GI_bid_hi_pp_Ghrelin_Totale['id_participant'])
mixed_model_fit = mixed_model.fit()



# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


## Ante Cibo

### basic stat

### GLP1

In [ ]:


# Define the formula
formula = 'bid ~ GLP_1_total + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_ac_GLP1, groups=df_GI_bid_hi_ac_GLP1['id_participant'], re_formula='1')
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")

### GLP-1 X type chx

In [ ]:


# Define the formula
formula = 'bid ~ GLP_1_total*type_chx + age + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_ac_GLP1, groups=df_GI_bid_hi_ac_GLP1['id_participant'], re_formula='1')
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")

### ghrelin Unacetylated

In [ ]:
# Define the formula
formula = 'bid ~ Ghrelin_Unacylated + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_ac_Ghrelin_Unacylated, groups=df_GI_bid_hi_ac_Ghrelin_Unacylated['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())


# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()


# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")

### Ghrelin Unacylated X type surgery

In [ ]:
# Define the formula
formula = 'bid ~ Ghrelin_Unacylated*type_chx + age + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_ac_Ghrelin_Unacylated, groups=df_GI_bid_hi_ac_Ghrelin_Unacylated['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())


# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()


# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")

### Ghrelin acetylated

In [ ]:
# Define the formula
formula = 'bid ~ Ghrelin_acylated + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_ac_Ghrelin_acylated, groups=df_GI_bid_hi_ac_Ghrelin_acylated['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")

### Ghrelin Acylated X surgery type

In [ ]:
# Define the formula
formula = 'bid ~ Ghrelin_acylated*type_chx + age + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_ac_Ghrelin_acylated, groups=df_GI_bid_hi_ac_Ghrelin_acylated['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")

### Leptin

In [ ]:
# Define the formula
formula = 'bid ~ Leptin + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_ac_Leptin, groups=df_GI_bid_hi_ac_Leptin['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()


# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()


# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")

### Leptin X type chx

In [ ]:
# Define the formula
formula = 'bid ~ Leptin*type_chx + age + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_ac_Leptin, groups=df_GI_bid_hi_ac_Leptin['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()


# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()


# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")

### PYY

In [ ]:
# Define the formula
formula = 'bid ~ PYY + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_ac_PYY, groups=df_GI_bid_hi_ac_PYY['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")

### Ghrelin Totale

#### Normality test for Ghrelin Totale with outliers

In [ ]:

# Assuming df_GI_bid_hi_ac_Ghrelin_Totale is already defined in the notebook

# Global normality test
ghrelin_totale_global = df_GI_bid_hi_ac_Ghrelin_Totale['Ghrelin_Totale']
shapiro_test_global = stats.shapiro(ghrelin_totale_global)
print(f"Global Shapiro-Wilk test p-value: {shapiro_test_global.pvalue}")

if shapiro_test_global.pvalue < 0.05:
    print("Ghrelin_Totale is not normally distributed globally.")
else:
    print("Ghrelin_Totale is normally distributed globally.")

# Normality test per session
for session, group in df_GI_bid_hi_ac_Ghrelin_Totale.groupby('session'):
    ghrelin_totale_session = group['Ghrelin_Totale']
    shapiro_test_session = stats.shapiro(ghrelin_totale_session)
    print(f"Session {session} Shapiro-Wilk test p-value: {shapiro_test_session.pvalue}")
    
    if shapiro_test_session.pvalue < 0.05:
        print(f"Ghrelin_Totale is not normally distributed for session {session}.")
    else:
        print(f"Ghrelin_Totale is normally distributed for session {session}.")

#### Normality test for log ghreline totale

In [ ]:
# Assuming df_GI_bid_hi_ac_Ghrelin_Totale is already defined in the notebook

# Global normality test for log_Ghrelin_Totale
log_ghrelin_totale_global = df_GI_bid_hi_ac_Ghrelin_Totale['log_Ghrelin_Totale']
shapiro_test_global_log = stats.shapiro(log_ghrelin_totale_global)
print(f"Global Shapiro-Wilk test p-value for log_Ghrelin_Totale: {shapiro_test_global_log.pvalue}")

if shapiro_test_global_log.pvalue < 0.05:
    print("log_Ghrelin_Totale is not normally distributed globally.")
else:
    print("log_Ghrelin_Totale is normally distributed globally.")

# Normality test per session for log_Ghrelin_Totale
for session, group in df_GI_bid_hi_ac_Ghrelin_Totale.groupby('session'):
    log_ghrelin_totale_session = group['log_Ghrelin_Totale']
    shapiro_test_session_log = stats.shapiro(log_ghrelin_totale_session)
    print(f"Session {session} Shapiro-Wilk test p-value for log_Ghrelin_Totale: {shapiro_test_session_log.pvalue}")
    
    if shapiro_test_session_log.pvalue < 0.05:
        print(f"log_Ghrelin_Totale is not normally distributed for session {session}.")
    else:
        print(f"log_Ghrelin_Totale is normally distributed for session {session}.")

In [ ]:
# Define the formula
formula = 'bid ~ Ghrelin_Totale + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_ac_Ghrelin_Totale, groups=df_GI_bid_hi_ac_Ghrelin_Totale['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")

### Ghreline Total Robust Regression

In [ ]:
# Define the formula
formula = 'bid ~ Ghrelin_Totale + age + type_chx + IMC_baseline + sexeF'

# Fit the robust regression model
robust_model = smf.rlm(formula, df_GI_bid_hi_ac_Ghrelin_Totale, M=sm.robust.norms.HuberT())
robust_model_fit = robust_model.fit()

# Print the summary of the model
print(robust_model_fit.summary())

# Residuals vs fitted values
residuals = robust_model_fit.resid
fitted_values = robust_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(robust_model_fit.resid, robust_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(robust_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = robust_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(robust_model_fit.model.exog, i) for i in range(robust_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(robust_model_fit.params)

# Number of observations
n = robust_model_fit.nobs

# Log-likelihood of the model
#log_likelihood = robust_model_fit.llf

# Calculate AIC
#aic = 2 * k - 2 * log_likelihood

# Calculate BIC
#bic = np.log(n) * k - 2 * log_likelihood

#print(f"AIC: {aic}")
#print(f"BIC: {bic}")

### Log Ghreline Totale

In [ ]:


# Define the formula
formula = 'bid ~ log_Ghrelin_Totale + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_ac_Ghrelin_Totale, groups=df_GI_bid_hi_ac_Ghrelin_Totale['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")

In [ ]:
min_ghrelin = 200
max_ghrelin = 340

log_min_ghrelin = np.log(min_ghrelin)
log_max_ghrelin = np.log(max_ghrelin)

print(f"Log of minimum Ghrelin_Totale (200 pg/ml): {log_min_ghrelin}")
print(f"Log of maximum Ghrelin_Totale (340 pg/ml): {log_max_ghrelin}")

coef_log_ghrelin = 0.345

change_in_bid = coef_log_ghrelin * (log_max_ghrelin - log_min_ghrelin)
print(f"Change in bid for Ghrelin_Totale range: {change_in_bid} dollars")

### Ghrelin Total X surgery type

In [ ]:
# Define the formula
formula = 'bid ~ Ghrelin_Totale*type_chx + age + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_ac_Ghrelin_Totale, groups=df_GI_bid_hi_ac_Ghrelin_Totale['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Announce that we are printing the Wald test results
print("Wald test results:")

# Perform a Wald test
w_matrix = np.eye(len(mixed_model_fit.params))  # Identity matrix for testing all fixed effects
wald_test = mixed_model_fit.wald_test_terms()

# Print the Wald test results with more precision for p-values
pd.set_option('display.float_format', lambda x: '%.10f' % x)
print(wald_test)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")

# Mixed models no outliers

## PP

### GLP-1 

In [ ]:

# Define the formula
formula = 'bid ~ GLP_1_total + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_GLP1_no_outliers_session, groups=df_GI_bid_hi_pp_GLP1_no_outliers_session['id_participant'], re_formula='1')
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


### GLP1 session 1 

In [ ]:



# Define the formula
formula = 'bid ~ GLP_1_total + type_chx + age+ IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_GLP1_no_outliers_session_ses_1, groups=df_GI_bid_hi_pp_GLP1_no_outliers_session_ses_1['id_participant'], re_formula='1')
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


### GLP1 Session 2 

In [ ]:



# Define the formula
formula = 'bid ~ GLP_1_total + type_chx + age+ IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_GLP1_no_outliers_session_ses_2, groups=df_GI_bid_hi_pp_GLP1_no_outliers_session_ses_2['id_participant'], re_formula='1')
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


### GLP1 session 3

In [ ]:



# Define the formula
formula = 'bid ~ GLP_1_total + type_chx + age+ IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_GLP1_no_outliers_session_ses_3, groups=df_GI_bid_hi_pp_GLP1_no_outliers_session_ses_3['id_participant'], re_formula='1')
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


### GLP1 session 4

In [ ]:



# Define the formula
formula = 'bid ~ GLP_1_total + type_chx + age+ IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_GLP1_no_outliers_session_ses_4, groups=df_GI_bid_hi_pp_GLP1_no_outliers_session_ses_4['id_participant'], re_formula='1')
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


### GLP1 X Session

In [ ]:

# Define the formula
formula = 'bid ~ GLP_1_total*session + age+ IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_GLP1_no_outliers_session, groups=df_GI_bid_hi_pp_GLP1_no_outliers_session['id_participant'], re_formula='1')
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


### GLP1 X  Type Surgery

In [ ]:

# Define the formula
formula = 'bid ~ GLP_1_total*type_chx + age+ IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_GLP1_no_outliers_session, groups=df_GI_bid_hi_pp_GLP1_no_outliers_session['id_participant'], re_formula='1')
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


### Ghrelin_Unacylated

In [ ]:
# Define the formula
formula = 'bid ~ Ghrelin_Unacylated + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_Ghrelin_Unacylated_no_outliers_session, groups=df_GI_bid_hi_pp_Ghrelin_Unacylated_no_outliers_session['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


### bid high vs type chx

In [ ]:

# Assuming df_GI_bid_hi_pp_no_outlier is already defined
plt.figure(figsize=(10, 6))

# Plotting the data
sns.lineplot(data=df_GI_bid_hi_pp_no_outlier, x='session', y='bid', hue='type_chx', marker='o')

# Adding title and labels
plt.title('Bid as a function of Session for Different Surgery Types')
plt.xlabel('Session')
plt.ylabel('Bid')
plt.legend(title='Surgery Type')

# Display the plot
plt.show()

### Ghrelin_acylated

In [ ]:
# Define the formula
formula = 'bid ~ Ghrelin_acylated + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_Ghrelin_acylated_no_outliers_session, groups=df_GI_bid_hi_pp_Ghrelin_acylated_no_outliers_session['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


### Ghrelin Totale

In [ ]:
# Define the formula
formula = 'bid ~ Ghrelin_Totale + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_Ghrelin_Totale_no_outliers_session, groups=df_GI_bid_hi_pp_Ghrelin_Totale_no_outliers_session['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


### Leptin

In [ ]:
# Define the formula
formula = 'bid ~ Leptin + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_Leptin_no_outliers_session, groups=df_GI_bid_hi_pp_Leptin_no_outliers_session['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()


# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()


# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


### PYY

In [ ]:
# Define the formula
formula = 'bid ~ PYY + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_PYY_no_outliers_session, groups=df_GI_bid_hi_pp_PYY_no_outliers_session['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()



# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


## Ante Cibo

### GLP1

In [ ]:
# Define the formula
formula = 'bid ~ GLP_1_total + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_ac_GLP1_no_outliers_session, groups=df_GI_bid_hi_ac_GLP1_no_outliers_session['id_participant'], re_formula='1')
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


### ghrelin Unacetylated

In [ ]:
# Define the formula
formula = 'bid ~ Ghrelin_Unacylated + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_ac_Ghrelin_Unacylated_no_outliers_session, groups=df_GI_bid_hi_ac_Ghrelin_Unacylated_no_outliers_session['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


### Ghrelin acetylated

In [ ]:
# Define the formula
formula = 'bid ~ Ghrelin_acylated + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_ac_Ghrelin_acylated_no_outliers_session, groups=df_GI_bid_hi_ac_Ghrelin_acylated_no_outliers_session['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


### Ghrelin Totale

In [ ]:
max_value = df_GI_bid_hi_ac_Ghrelin_Totale_no_outliers_session['Ghrelin_Totale'].max()
print(max_value)

### Graph au propre GTot vs bid high

In [ ]:
# Calculate the average Ghrelin_Totale per session
average_ghrelin = df_GI_bid_hi_ac_Ghrelin_Totale_no_outliers_session.groupby('session')['Ghrelin_Totale'].mean()

# Calculate the average bid per session
average_bid = df_GI_bid_hi_ac_Ghrelin_Totale_no_outliers_session.groupby('session')['bid'].mean()

# Plotting average Ghrelin_Totale per session with labels
fig, ax1 = plt.subplots(figsize=(10, 6))

# Primary y-axis for Ghrelin_Totale
ax1.plot(average_ghrelin.index, average_ghrelin.values, marker='o', linestyle='-', color='b', label='Ghréline Totale')
ax1.set_xlabel('Session')
ax1.set_ylabel('Ghrelin Totale (pg/ml)', color='black')
ax1.tick_params(axis='y', labelcolor='black')
ax1.grid(True)
ax1.set_ylim(0, 930)

# Ensure y-axis ticks are integers
ax1.yaxis.set_major_locator(plt.MaxNLocator(integer=True))

# Set custom x-axis labels
ax1.set_xticks(average_ghrelin.index)
ax1.set_xticklabels(['pré-chirurgie', '4 mois', '12 mois', '24 mois'])

# Secondary y-axis for bid
ax2 = ax1.twinx()
ax2.plot(average_bid.index, average_bid.values, marker='o', linestyle='-', color='orange', label='Mises pour aliments riches en calories')
ax2.set_ylabel('Mises ($)', color='black')
ax2.tick_params(axis='y', labelcolor='black')
ax2.set_ylim(0, 5)

# Calculate the standard deviation of bid per session
std_bid = df_GI_bid_hi_ac_Ghrelin_Totale_no_outliers_session.groupby('session')['bid'].std()

# Adding labels for average_bid
for i, (avg, std) in enumerate(zip(average_bid.values, std_bid.values)):
    x_coord = i  # Use the index position directly for alignment
    if i == 0:  # Session 1
        ax2.text(x_coord + 0.2, avg + 0.3, f'{avg:.2f} ± {std:.2f}', ha='center', va='top', color='orange')  # Above and centered
    else:  # Sessions 2, 3, 4
        ax2.text(x_coord - 0.1, avg + 0.4, f'{avg:.2f} ± {std:.2f}', ha='center', va='top', color='orange')  # Below and centered
# Calculate the standard deviation of Ghrelin_Totale per session
std_ghrelin = df_GI_bid_hi_ac_Ghrelin_Totale_no_outliers_session.groupby('session')['Ghrelin_Totale'].std()

# Adding labels for average_ghrelin
for i, (avg, std) in enumerate(zip(average_ghrelin.values, std_ghrelin.values)):
    x_coord = i  # Use the index position directly for alignment
    if i == 0:  # Session 1
        ax1.text(x_coord + 0.2, avg - 50, f'{avg:.2f} ± {std:.2f}', ha='center', va='top', color='blue')  # Above and centered
    else:  # Sessions 2, 3, 4
        ax1.text(x_coord - 0.1, avg - 50 , f'{avg:.2f} ± {std:.2f}', ha='center', va='bottom', color='blue')  # Below and centered

# Adding legends
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

ax1.grid(False)
plt.grid(False)

plt.show()

### Bid = Ghreline totale + confounds

In [ ]:
# Define the formula
formula = 'bid ~ Ghrelin_Totale + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_ac_Ghrelin_Totale_no_outliers_session, groups=df_GI_bid_hi_ac_Ghrelin_Totale_no_outliers_session['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

# Perform a Wald test
print("Wald test results:")
w_matrix = np.eye(len(mixed_model_fit.params))  # Identity matrix for testing all fixed effects
wald_test = mixed_model_fit.wald_test_terms()

# Print the Wald test results with more precision for p-values
pd.set_option('display.float_format', lambda x: '%.10f' % x)
print(wald_test)

print(f"AIC: {aic}")
print(f"BIC: {bic}")


### Ghrelin total X type Chx

In [ ]:
# Define the formula
formula = 'bid ~ Ghrelin_Totale*type_chx + age + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_ac_Ghrelin_Totale_no_outliers_session, groups=df_GI_bid_hi_ac_Ghrelin_Totale_no_outliers_session['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Announce that we are printing the Wald test results
print("Wald test results:")

# Perform a Wald test
w_matrix = np.eye(len(mixed_model_fit.params))  # Identity matrix for testing all fixed effects
wald_test = mixed_model_fit.wald_test_terms()

# Print the Wald test results with more precision for p-values
pd.set_option('display.float_format', lambda x: '%.10f' % x)
print(wald_test)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


### Leptin

In [ ]:
# Define the formula
formula = 'bid ~ Leptin + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_ac_Leptin_no_outliers_session, groups=df_GI_bid_hi_ac_Leptin_no_outliers_session['id_participant'])
mixed_model_fit = mixed_model.fit()

# Print the summary of the model
print(mixed_model_fit.summary())

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


### PYY

In [ ]:
# Define the formula
formula = 'bid ~ PYY + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_ac_PYY_no_outliers_session, groups=df_GI_bid_hi_ac_PYY_no_outliers_session['id_participant'])
mixed_model_fit = mixed_model.fit()

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# Print the summary of the model
print(mixed_model_fit.summary())

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


# make sure bid findings are the same for hormones subsample v.s. bid total sample

### Average bid plot hi

In [ ]:
from matplotlib.ticker import FuncFormatter

df = df_GI_bid_hi_pp_GLP1_no_outliers_session

# Initialize the DataFrame with the specified columns
results = pd.DataFrame(columns=['Pré-Chirurgie', '4 mois', '12 mois', '24 mois', 'Valeur P'])

# Count the unique id_participant for session = 1,2,3,4
unique_participants_per_session = df.groupby('session')['id_participant'].nunique()

# Extract the number of unique participants for each session
N_pre_chirurgie = unique_participants_per_session.get('1', 0)
N_4_mois = unique_participants_per_session.get('2', 0)
N_12_mois = unique_participants_per_session.get('3', 0)
N_24_mois = unique_participants_per_session.get('4', 0)

# Calculate the mean and standard deviation of age for each session
mean_age_per_session = df.groupby('session')['age'].mean()
std_age_per_session = df.groupby('session')['age'].std()

# Extract the mean and standard deviation of age for each session
mean_std_age_pre_chirurgie = f"{mean_age_per_session.get('1', 0):.2f} ± {std_age_per_session.get('1', 0):.2f}"
mean_std_age_4_mois = f"{mean_age_per_session.get('2', 0):.2f} ± {std_age_per_session.get('2', 0):.2f}"
mean_std_age_12_mois = f"{mean_age_per_session.get('3', 0):.2f} ± {std_age_per_session.get('3', 0):.2f}"
mean_std_age_24_mois = f"{mean_age_per_session.get('4', 0):.2f} ± {std_age_per_session.get('4', 0):.2f}"
# Count the number of sexeF = 1 per unique id_participant for each session
sexeF_counts_per_session = df[df['sexeF'] == 1].drop_duplicates(subset=['session', 'id_participant']).groupby('session')['id_participant'].nunique()

# Count the number of sexeF = 0 per unique id_participant for each session
sexeM_counts_per_session = df[df['sexeF'] == 0].drop_duplicates(subset=['session', 'id_participant']).groupby('session')['id_participant'].nunique()

# Create the variables for each session
sexe_pre_chirurgie = {0: sexeM_counts_per_session.get('1', 0), 1: sexeF_counts_per_session.get('1', 0)}
sexe_4_mois = {0: sexeM_counts_per_session.get('2', 0), 1: sexeF_counts_per_session.get('2', 0)}
sexe_12_mois = {0: sexeM_counts_per_session.get('3', 0), 1: sexeF_counts_per_session.get('3', 0)}
sexe_24_mois = {0: sexeM_counts_per_session.get('4', 0), 1: sexeF_counts_per_session.get('4', 0)}

# Count the number of times type_chx = 1 per unique id_participant for each session
gastrectomie_counts_per_session = df[df['type_chx'] == 1].drop_duplicates(subset=['session', 'id_participant']).groupby(['session', 'id_participant']).size().unstack(fill_value=0)

gastrectomie_pre_chirurgie = gastrectomie_counts_per_session.loc['1'].sum() if '1' in gastrectomie_counts_per_session.index else 0
gastrectomie_4_mois = gastrectomie_counts_per_session.loc['2'].sum() if '2' in gastrectomie_counts_per_session.index else 0
gastrectomie_12_mois = gastrectomie_counts_per_session.loc['3'].sum() if '3' in gastrectomie_counts_per_session.index else 0
gastrectomie_24_mois = gastrectomie_counts_per_session.loc['4'].sum() if '4' in gastrectomie_counts_per_session.index else 0

# Count the number of times type_chx = 2 per unique id_participant for each session
rygb_counts_per_session = df[df['type_chx'] == 2].drop_duplicates(subset=['session', 'id_participant']).groupby(['session', 'id_participant']).size().unstack(fill_value=0)

rygb_pre_chirurgie = rygb_counts_per_session.loc['1'].sum() if '1' in rygb_counts_per_session.index else 0
rygb_4_mois = rygb_counts_per_session.loc['2'].sum() if '2' in rygb_counts_per_session.index else 0
rygb_12_mois = rygb_counts_per_session.loc['3'].sum() if '3' in rygb_counts_per_session.index else 0
rygb_24_mois = rygb_counts_per_session.loc['4'].sum() if '4' in rygb_counts_per_session.index else 0

# Count the number of times type_chx = 3 per unique id_participant for each session
dbp_sd_counts_per_session = df[df['type_chx'] == 3].drop_duplicates(subset=['session', 'id_participant']).groupby(['session', 'id_participant']).size().unstack(fill_value=0)

dbp_sd_pre_chirurgie = dbp_sd_counts_per_session.loc['1'].sum() if '1' in dbp_sd_counts_per_session.index else 0
dbp_sd_4_mois = dbp_sd_counts_per_session.loc['2'].sum() if '2' in dbp_sd_counts_per_session.index else 0
dbp_sd_12_mois = dbp_sd_counts_per_session.loc['3'].sum() if '3' in dbp_sd_counts_per_session.index else 0
dbp_sd_24_mois = dbp_sd_counts_per_session.loc['4'].sum() if '4' in dbp_sd_counts_per_session.index else 0

# Calculate the mean and standard deviation of imc_recherche for each session
mean_imc_per_session = df.groupby('session')['imc_recherche_x'].mean()
std_imc_per_session = df.groupby('session')['imc_recherche_x'].std()

# Extract the mean and standard deviation of imc_recherche for each session
mean_std_imc_pre_chirurgie = f"{mean_imc_per_session.get('1', 0):.2f} ± {std_imc_per_session.get('1', 0):.2f}"
mean_std_imc_4_mois = f"{mean_imc_per_session.get('2', 0):.2f} ± {std_imc_per_session.get('2', 0):.2f}"
mean_std_imc_12_mois = f"{mean_imc_per_session.get('3', 0):.2f} ± {std_imc_per_session.get('3', 0):.2f}"
mean_std_imc_24_mois = f"{mean_imc_per_session.get('4', 0):.2f} ± {std_imc_per_session.get('4', 0):.2f}"

# Calculate the mean and standard deviation of twl_recherche_x for each session
mean_twl_per_session = df.groupby('session')['twl_recherche_x'].mean() * 100
std_twl_per_session = df.groupby('session')['twl_recherche_x'].std() * 100

# Extract the mean and standard deviation of twl_recherche_x for each session
mean_std_twl_pre_chirurgie = f"{mean_twl_per_session.get('1', 0):.2f} ± {std_twl_per_session.get('1', 0):.2f}"
mean_std_twl_4_mois = f"{mean_twl_per_session.get('2', 0):.2f} ± {std_twl_per_session.get('2', 0):.2f}"
mean_std_twl_12_mois = f"{mean_twl_per_session.get('3', 0):.2f} ± {std_twl_per_session.get('3', 0):.2f}"
mean_std_twl_24_mois = f"{mean_twl_per_session.get('4', 0):.2f} ± {std_twl_per_session.get('4', 0):.2f}"



# Calculate the mean and standard error of the mean (SEM) of bid for each session
mean_bid_per_session = df.groupby('session')['bid'].mean()
sem_bid_per_session = df.groupby('session')['bid'].sem()
mean_std_bid_per_session = df.groupby('session')['bid'].std()

# Extract the mean and standard deviation of bid for each session
mean_std_bid_pre_chirurgie = f"{mean_bid_per_session.get('1', 0):.2f} ± {mean_std_bid_per_session.get('1', 0):.2f}"
mean_std_bid_4_mois = f"{mean_bid_per_session.get('2', 0):.2f} ± {mean_std_bid_per_session.get('2', 0):.2f}"
mean_std_bid_12_mois = f"{mean_bid_per_session.get('3', 0):.2f} ± {mean_std_bid_per_session.get('3', 0):.2f}"
mean_std_bid_24_mois = f"{mean_bid_per_session.get('4', 0):.2f} ± {mean_std_bid_per_session.get('4', 0):.2f}"

# Extract the mean and SEM of bid for each session
mean_sem_bid_pre_chirurgie = f"{mean_bid_per_session.get('1', 0):.2f} ± {sem_bid_per_session.get('1', 0):.2f}"
mean_sem_bid_4_mois = f"{mean_bid_per_session.get('2', 0):.2f} ± {sem_bid_per_session.get('2', 0):.2f}"
mean_sem_bid_12_mois = f"{mean_bid_per_session.get('3', 0):.2f} ± {sem_bid_per_session.get('3', 0):.2f}"
mean_sem_bid_24_mois = f"{mean_bid_per_session.get('4', 0):.2f} ± {sem_bid_per_session.get('4', 0):.2f}"

# Calculate the mean and SEM of bid for each session where calories = 'hi'
mean_bid_high_per_session = df[df['calories'] == 'hi'].groupby('session')['bid'].mean()
sem_bid_high_per_session = df[df['calories'] == 'hi'].groupby('session')['bid'].sem()

# Calculate the standard deviation of bid for each session where calories = 'hi'
std_bid_high_per_session = df[df['calories'] == 'hi'].groupby('session')['bid'].std()
# Extract the mean and standard deviation of bid for each session where calories = 'hi'
mean_std_bid_high_pre_chirurgie = f"{mean_bid_high_per_session.get('1', 0):.2f} ± {std_bid_high_per_session.get('1', 0):.2f}"
mean_std_bid_high_4_mois = f"{mean_bid_high_per_session.get('2', 0):.2f} ± {std_bid_high_per_session.get('2', 0):.2f}"
mean_std_bid_high_12_mois = f"{mean_bid_high_per_session.get('3', 0):.2f} ± {std_bid_high_per_session.get('3', 0):.2f}"
mean_std_bid_high_24_mois = f"{mean_bid_high_per_session.get('4', 0):.2f} ± {std_bid_high_per_session.get('4', 0):.2f}"

# Extract the mean and SEM of bid for each session where calories = 'hi'
mean_sem_bid_high_pre_chirurgie = f"{mean_bid_high_per_session.get('1', 0):.2f} ± {sem_bid_high_per_session.get('1', 0):.2f}"
mean_sem_bid_high_4_mois = f"{mean_bid_high_per_session.get('2', 0):.2f} ± {sem_bid_high_per_session.get('2', 0):.2f}"
mean_sem_bid_high_12_mois = f"{mean_bid_high_per_session.get('3', 0):.2f} ± {sem_bid_high_per_session.get('3', 0):.2f}"
mean_sem_bid_high_24_mois = f"{mean_bid_high_per_session.get('4', 0):.2f} ± {sem_bid_high_per_session.get('4', 0):.2f}"
print(sem_bid_high_per_session)


# Calculate the mean and SEM of bid for each session where calories = 'lo'
mean_bid_low_per_session = df[df['calories'] == 'lo'].groupby('session')['bid'].mean()
sem_bid_low_per_session = df[df['calories'] == 'lo'].groupby('session')['bid'].sem()

# Calculate the standard deviation of bid for each session where calories = 'lo'

std_bid_low_per_session = df[df['calories'] == 'lo'].groupby('session')['bid'].std()
print(sem_bid_low_per_session)

# Extract the mean and standard deviation of bid for each session where calories = 'lo'
mean_std_bid_low_pre_chirurgie = f"{mean_bid_low_per_session.get('1', 0):.2f} ± {std_bid_low_per_session.get('1', 0):.2f}"
mean_std_bid_low_4_mois = f"{mean_bid_low_per_session.get('2', 0):.2f} ± {std_bid_low_per_session.get('2', 0):.2f}"
mean_std_bid_low_12_mois = f"{mean_bid_low_per_session.get('3', 0):.2f} ± {std_bid_low_per_session.get('3', 0):.2f}"
mean_std_bid_low_24_mois = f"{mean_bid_low_per_session.get('4', 0):.2f} ± {std_bid_low_per_session.get('4', 0):.2f}"

# Extract the mean and SEM of bid for each session where calories = 'lo'
mean_sem_bid_low_pre_chirurgie = f"{mean_bid_low_per_session.get('1', 0):.2f} ± {sem_bid_low_per_session.get('1', 0):.2f}"
mean_sem_bid_low_4_mois = f"{mean_bid_low_per_session.get('2', 0):.2f} ± {sem_bid_low_per_session.get('2', 0):.2f}"
mean_sem_bid_low_12_mois = f"{mean_bid_low_per_session.get('3', 0):.2f} ± {sem_bid_low_per_session.get('3', 0):.2f}"
mean_sem_bid_low_24_mois = f"{mean_bid_low_per_session.get('4', 0):.2f} ± {sem_bid_low_per_session.get('4', 0):.2f}"

# Define the sessions
sessions = ['Pré-Chirurgie', '4 mois', '12 mois', '24 mois']

# Define the means and standard errors
mean_bid_low = [mean_bid_low_per_session.get(str(i), 0) for i in range(1, 5)]
sem_bid_low = [sem_bid_low_per_session.get(str(i), 0) for i in range(1, 5)]
mean_bid_high = [mean_bid_high_per_session.get(str(i), 0) for i in range(1, 5)]
sem_bid_high = [sem_bid_high_per_session.get(str(i), 0) for i in range(1, 5)]

# Plot the data
plt.figure(figsize=(10, 6))
plt.errorbar(sessions, mean_bid_low, yerr=sem_bid_low, label='WTP moyen faible', marker='o', capsize=5)
plt.errorbar(sessions, mean_bid_high, yerr=sem_bid_high, label='WTP moyen riche', marker='o', capsize=5)
plt.ylim(0, 5)
# Add labels for each point
for i, session in enumerate(sessions):
    if session == 'Pré-Chirurgie':
        plt.text(i-0.1, mean_bid_low[i] - sem_bid_low[i] + 0.2, f"{mean_bid_low[i]:.2f}".replace('.', ',') + " ± " + f"{sem_bid_low[i]:.2f}".replace('.', ','), ha='left', color='blue',fontsize=20)
        plt.text(i-0.1, mean_bid_high[i] - sem_bid_high[i] - 0.6, f"{mean_bid_high[i]:.2f}".replace('.', ',') + " ± " + f"{sem_bid_high[i]:.2f}".replace('.', ','), ha='left', color='orange',fontsize=20)
    else:
        plt.text(i, mean_bid_low[i] + sem_bid_low[i] + 0.1, f"{mean_bid_low[i]:.2f}".replace('.', ',') + " ± " + f"{sem_bid_low[i]:.2f}".replace('.', ','), ha='center', color='blue',fontsize=20)
        plt.text(i, mean_bid_high[i] + sem_bid_high[i] - 0.4, f"{mean_bid_high[i]:.2f}".replace('.', ',') + " ± " + f"{sem_bid_high[i]:.2f}".replace('.', ','), ha='center', color='orange',fontsize=20)

# Add labels and title
plt.ylabel('Valeur moyenne des mises ($)',fontsize=20)
plt.xticks(fontsize=20)

formatter = FuncFormatter(lambda x, _: f'{x:.2f}')
plt.yticks(fontsize=20)
plt.gca().tick_params(axis='x', pad=20)
plt.gca().yaxis.set_major_formatter(formatter)
plt.legend(['Faibles en calories', 'Riches en calories'], fontsize=20)
plt.grid(False)
plt.show()


# Add the mean and standard deviation of age to the results DataFrame
results.loc['N'] = [N_pre_chirurgie, N_4_mois, N_12_mois, N_24_mois, None]
results.loc['Age (mean ± std)'] = [mean_std_age_pre_chirurgie, mean_std_age_4_mois, mean_std_age_12_mois, mean_std_age_24_mois, None]
# Format the counts as "count of 0 : count of 1"
format_sexe_counts = lambda counts: f"{counts.get(0, 0)} : {counts.get(1, 0)}"
results.loc['Sexe (M:F)'] = [format_sexe_counts(sexe_pre_chirurgie), format_sexe_counts(sexe_4_mois), format_sexe_counts(sexe_12_mois), format_sexe_counts(sexe_24_mois), None]
# Add the gastrectomie counts to the results DataFrame
results.loc['Gastrectomie (n)'] = [gastrectomie_pre_chirurgie, gastrectomie_4_mois, gastrectomie_12_mois, gastrectomie_24_mois, None]
# Add the RYGB counts to the results DataFrame
results.loc['RYGB (n)'] = [rygb_pre_chirurgie, rygb_4_mois, rygb_12_mois, rygb_24_mois, None]

# Add the DBP-SD counts to the results DataFrame
results.loc['DBP-SD (n)'] = [dbp_sd_pre_chirurgie, dbp_sd_4_mois, dbp_sd_12_mois, dbp_sd_24_mois, None]

# Add the mean and standard deviation of imc_recherche to the results DataFrame
results.loc['IMC (kg/m2) (moy. ± et.)'] = [mean_std_imc_pre_chirurgie, mean_std_imc_4_mois, mean_std_imc_12_mois, mean_std_imc_24_mois, None]

# Add the mean and standard deviation of twl_recherche_x to the results DataFrame
results.loc['Pourcentage de perte de poids (%)'] = [mean_std_twl_pre_chirurgie, mean_std_twl_4_mois, mean_std_twl_12_mois, mean_std_twl_24_mois, None]

# Add the mean and standard deviation of bid to the results DataFrame
results.loc['Mise (moy. ± et.)'] = [mean_std_bid_pre_chirurgie, mean_std_bid_4_mois, mean_std_bid_12_mois, mean_std_bid_24_mois, None]

# Add the mean and standard deviation of bid to the results DataFrame
results.loc['Mise (élevé) (moy. ± et.)'] = [mean_std_bid_high_pre_chirurgie, mean_std_bid_high_4_mois, mean_std_bid_high_12_mois, mean_std_bid_high_24_mois, None]

# Add the mean and standard deviation of bid for low calories to the results DataFrame
results.loc['Mise (faible) (moy. ± et.)'] = [mean_std_bid_low_pre_chirurgie, mean_std_bid_low_4_mois, mean_std_bid_low_12_mois, mean_std_bid_low_24_mois, None]

print(results)


### Mixed model bid vs session for hormones sub-sample

In [ ]:
# Define the formula
formula = 'bid ~ session + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_hi_pp_PYY, groups=df_GI_bid_hi_pp_PYY['id_participant'])
mixed_model_fit = mixed_model.fit()

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# Print the summary of the model
print(mixed_model_fit.summary())

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


### Mixed model bid = type_chx X session

In [ ]:
# Define the formula
formula = 'bid ~ session + type_chx'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, bid_avg_hi, groups=bid_avg_hi['id_participant'],re_formula = '1 + type_chx')
mixed_model_fit = mixed_model.fit()

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# Print the summary of the model
print(mixed_model_fit.summary())

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


### Make sure there is no variation in bid lo

### Create GI bid lo dfs

In [ ]:
df_GI_bid_lo_ac = df_GI_bid.copy()
df_GI_bid_lo_ac = df_GI_bid_lo_ac[(df_GI_bid_lo_ac['fed_state'] == 'ac') & (df_GI_bid_lo_ac['calories'] == 'lo')]

df_GI_bid_lo_pp = df_GI_bid.copy()
df_GI_bid_lo_pp = df_GI_bid_lo_pp[(df_GI_bid_lo_pp['fed_state'] == 'pp') & (df_GI_bid_lo_pp['calories'] == 'lo')]
df_GI_bid_lo_pp

### Create dfs for bid lo and PYY

In [ ]:
# Find participants and sessions where PYY is not recorded (PP)
df_no_PYY_pp = df_GI_bid_lo_pp[df_GI_bid_lo_pp['PYY'].isna()][['id_participant', 'session']].drop_duplicates()
print(df_no_PYY_pp)

# Create a DF where there are no Nans for PYY PP
df_GI_bid_lo_pp_PYY = df_GI_bid_lo_pp.dropna(subset=['PYY'])

# Calculate z-scores for PYY and session
df_GI_bid_lo_pp_PYY['PYY_zscore_session'] = df_GI_bid_lo_pp_PYY.groupby('session')['PYY'].transform(lambda x: (x - x.mean()) / x.std())
df_GI_bid_lo_pp_PYY['PYY_zscore'] = zscore(df_GI_bid_lo_pp_PYY['PYY'])
print(df_GI_bid_lo_pp_PYY['PYY_zscore_session'])

# Print PYY_zscore_session and PYY_zscore
print(df_GI_bid_lo_pp_PYY[['PYY_zscore_session', 'PYY_zscore']])

print(basic_stat(df_GI_bid_lo_pp_PYY))

# Find outliers based on PYY zscore
outliers_PYY_pp = df_GI_bid_lo_pp_PYY[(df_GI_bid_lo_pp_PYY['PYY_zscore'] > 3) | (df_GI_bid_lo_pp_PYY['PYY_zscore'] < -3)]
print(outliers_PYY_pp[['id_participant', 'session']])

# Find outliers based on PYY zscore per session
outliers_PYY_session_pp = df_GI_bid_lo_pp_PYY[(df_GI_bid_lo_pp_PYY['PYY_zscore_session'] > 3) | (df_GI_bid_lo_pp_PYY['PYY_zscore_session'] < -3)]
print(outliers_PYY_session_pp[['id_participant', 'session']])

# Remove outliers based on z-score threshold of 3
df_GI_bid_lo_pp_PYY_no_outliers = df_GI_bid_lo_pp_PYY[df_GI_bid_lo_pp_PYY['PYY_zscore'].abs() <= 3]
print(basic_stat(df_GI_bid_lo_pp_PYY_no_outliers))

# Remove outliers based on z-score threshold of 3 for session
df_GI_bid_lo_pp_PYY_no_outliers_session = df_GI_bid_lo_pp_PYY[df_GI_bid_lo_pp_PYY['PYY_zscore_session'].abs() <= 3]
print(basic_stat(df_GI_bid_lo_pp_PYY_no_outliers_session))

### Plot bid lo

In [ ]:

df = df_GI_bid_lo_pp_PYY

# Initialize the DataFrame with the specified columns
results = pd.DataFrame(columns=['Pré-Chirurgie', '4 mois', '12 mois', '24 mois', 'Valeur P'])

# Count the unique id_participant for session = 1,2,3,4
unique_participants_per_session = df.groupby('session')['id_participant'].nunique()

# Extract the number of unique participants for each session
N_pre_chirurgie = unique_participants_per_session.get('1', 0)
N_4_mois = unique_participants_per_session.get('2', 0)
N_12_mois = unique_participants_per_session.get('3', 0)
N_24_mois = unique_participants_per_session.get('4', 0)

# Calculate the mean and standard deviation of age for each session
mean_age_per_session = df.groupby('session')['age'].mean()
std_age_per_session = df.groupby('session')['age'].std()

# Extract the mean and standard deviation of age for each session
mean_std_age_pre_chirurgie = f"{mean_age_per_session.get('1', 0):.2f} ± {std_age_per_session.get('1', 0):.2f}"
mean_std_age_4_mois = f"{mean_age_per_session.get('2', 0):.2f} ± {std_age_per_session.get('2', 0):.2f}"
mean_std_age_12_mois = f"{mean_age_per_session.get('3', 0):.2f} ± {std_age_per_session.get('3', 0):.2f}"
mean_std_age_24_mois = f"{mean_age_per_session.get('4', 0):.2f} ± {std_age_per_session.get('4', 0):.2f}"
# Count the number of sexeF = 1 per unique id_participant for each session
sexeF_counts_per_session = df[df['sexeF'] == 1].drop_duplicates(subset=['session', 'id_participant']).groupby('session')['id_participant'].nunique()

# Count the number of sexeF = 0 per unique id_participant for each session
sexeM_counts_per_session = df[df['sexeF'] == 0].drop_duplicates(subset=['session', 'id_participant']).groupby('session')['id_participant'].nunique()

# Create the variables for each session
sexe_pre_chirurgie = {0: sexeM_counts_per_session.get('1', 0), 1: sexeF_counts_per_session.get('1', 0)}
sexe_4_mois = {0: sexeM_counts_per_session.get('2', 0), 1: sexeF_counts_per_session.get('2', 0)}
sexe_12_mois = {0: sexeM_counts_per_session.get('3', 0), 1: sexeF_counts_per_session.get('3', 0)}
sexe_24_mois = {0: sexeM_counts_per_session.get('4', 0), 1: sexeF_counts_per_session.get('4', 0)}

# Count the number of times type_chx = 1 per unique id_participant for each session
gastrectomie_counts_per_session = df[df['type_chx'] == 1].drop_duplicates(subset=['session', 'id_participant']).groupby(['session', 'id_participant']).size().unstack(fill_value=0)

gastrectomie_pre_chirurgie = gastrectomie_counts_per_session.loc['1'].sum() if '1' in gastrectomie_counts_per_session.index else 0
gastrectomie_4_mois = gastrectomie_counts_per_session.loc['2'].sum() if '2' in gastrectomie_counts_per_session.index else 0
gastrectomie_12_mois = gastrectomie_counts_per_session.loc['3'].sum() if '3' in gastrectomie_counts_per_session.index else 0
gastrectomie_24_mois = gastrectomie_counts_per_session.loc['4'].sum() if '4' in gastrectomie_counts_per_session.index else 0

# Count the number of times type_chx = 2 per unique id_participant for each session
rygb_counts_per_session = df[df['type_chx'] == 2].drop_duplicates(subset=['session', 'id_participant']).groupby(['session', 'id_participant']).size().unstack(fill_value=0)

rygb_pre_chirurgie = rygb_counts_per_session.loc['1'].sum() if '1' in rygb_counts_per_session.index else 0
rygb_4_mois = rygb_counts_per_session.loc['2'].sum() if '2' in rygb_counts_per_session.index else 0
rygb_12_mois = rygb_counts_per_session.loc['3'].sum() if '3' in rygb_counts_per_session.index else 0
rygb_24_mois = rygb_counts_per_session.loc['4'].sum() if '4' in rygb_counts_per_session.index else 0

# Count the number of times type_chx = 3 per unique id_participant for each session
dbp_sd_counts_per_session = df[df['type_chx'] == 3].drop_duplicates(subset=['session', 'id_participant']).groupby(['session', 'id_participant']).size().unstack(fill_value=0)

dbp_sd_pre_chirurgie = dbp_sd_counts_per_session.loc['1'].sum() if '1' in dbp_sd_counts_per_session.index else 0
dbp_sd_4_mois = dbp_sd_counts_per_session.loc['2'].sum() if '2' in dbp_sd_counts_per_session.index else 0
dbp_sd_12_mois = dbp_sd_counts_per_session.loc['3'].sum() if '3' in dbp_sd_counts_per_session.index else 0
dbp_sd_24_mois = dbp_sd_counts_per_session.loc['4'].sum() if '4' in dbp_sd_counts_per_session.index else 0

# Calculate the mean and standard deviation of imc_recherche for each session
mean_imc_per_session = df.groupby('session')['imc_recherche_x'].mean()
std_imc_per_session = df.groupby('session')['imc_recherche_x'].std()

# Extract the mean and standard deviation of imc_recherche for each session
mean_std_imc_pre_chirurgie = f"{mean_imc_per_session.get('1', 0):.2f} ± {std_imc_per_session.get('1', 0):.2f}"
mean_std_imc_4_mois = f"{mean_imc_per_session.get('2', 0):.2f} ± {std_imc_per_session.get('2', 0):.2f}"
mean_std_imc_12_mois = f"{mean_imc_per_session.get('3', 0):.2f} ± {std_imc_per_session.get('3', 0):.2f}"
mean_std_imc_24_mois = f"{mean_imc_per_session.get('4', 0):.2f} ± {std_imc_per_session.get('4', 0):.2f}"

# Calculate the mean and standard deviation of twl_recherche_x for each session
mean_twl_per_session = df.groupby('session')['twl_recherche_x'].mean() * 100
std_twl_per_session = df.groupby('session')['twl_recherche_x'].std() * 100

# Extract the mean and standard deviation of twl_recherche_x for each session
mean_std_twl_pre_chirurgie = f"{mean_twl_per_session.get('1', 0):.2f} ± {std_twl_per_session.get('1', 0):.2f}"
mean_std_twl_4_mois = f"{mean_twl_per_session.get('2', 0):.2f} ± {std_twl_per_session.get('2', 0):.2f}"
mean_std_twl_12_mois = f"{mean_twl_per_session.get('3', 0):.2f} ± {std_twl_per_session.get('3', 0):.2f}"
mean_std_twl_24_mois = f"{mean_twl_per_session.get('4', 0):.2f} ± {std_twl_per_session.get('4', 0):.2f}"



# Calculate the mean and standard error of the mean (SEM) of bid for each session
mean_bid_per_session = df.groupby('session')['bid'].mean()
sem_bid_per_session = df.groupby('session')['bid'].sem()
mean_std_bid_per_session = df.groupby('session')['bid'].std()

# Extract the mean and standard deviation of bid for each session
mean_std_bid_pre_chirurgie = f"{mean_bid_per_session.get('1', 0):.2f} ± {mean_std_bid_per_session.get('1', 0):.2f}"
mean_std_bid_4_mois = f"{mean_bid_per_session.get('2', 0):.2f} ± {mean_std_bid_per_session.get('2', 0):.2f}"
mean_std_bid_12_mois = f"{mean_bid_per_session.get('3', 0):.2f} ± {mean_std_bid_per_session.get('3', 0):.2f}"
mean_std_bid_24_mois = f"{mean_bid_per_session.get('4', 0):.2f} ± {mean_std_bid_per_session.get('4', 0):.2f}"

# Extract the mean and SEM of bid for each session
mean_sem_bid_pre_chirurgie = f"{mean_bid_per_session.get('1', 0):.2f} ± {sem_bid_per_session.get('1', 0):.2f}"
mean_sem_bid_4_mois = f"{mean_bid_per_session.get('2', 0):.2f} ± {sem_bid_per_session.get('2', 0):.2f}"
mean_sem_bid_12_mois = f"{mean_bid_per_session.get('3', 0):.2f} ± {sem_bid_per_session.get('3', 0):.2f}"
mean_sem_bid_24_mois = f"{mean_bid_per_session.get('4', 0):.2f} ± {sem_bid_per_session.get('4', 0):.2f}"

# Calculate the mean and SEM of bid for each session where calories = 'hi'
mean_bid_high_per_session = df[df['calories'] == 'hi'].groupby('session')['bid'].mean()
sem_bid_high_per_session = df[df['calories'] == 'hi'].groupby('session')['bid'].sem()

# Calculate the standard deviation of bid for each session where calories = 'hi'
std_bid_high_per_session = df[df['calories'] == 'hi'].groupby('session')['bid'].std()
# Extract the mean and standard deviation of bid for each session where calories = 'hi'
mean_std_bid_high_pre_chirurgie = f"{mean_bid_high_per_session.get('1', 0):.2f} ± {std_bid_high_per_session.get('1', 0):.2f}"
mean_std_bid_high_4_mois = f"{mean_bid_high_per_session.get('2', 0):.2f} ± {std_bid_high_per_session.get('2', 0):.2f}"
mean_std_bid_high_12_mois = f"{mean_bid_high_per_session.get('3', 0):.2f} ± {std_bid_high_per_session.get('3', 0):.2f}"
mean_std_bid_high_24_mois = f"{mean_bid_high_per_session.get('4', 0):.2f} ± {std_bid_high_per_session.get('4', 0):.2f}"

# Extract the mean and SEM of bid for each session where calories = 'hi'
mean_sem_bid_high_pre_chirurgie = f"{mean_bid_high_per_session.get('1', 0):.2f} ± {sem_bid_high_per_session.get('1', 0):.2f}"
mean_sem_bid_high_4_mois = f"{mean_bid_high_per_session.get('2', 0):.2f} ± {sem_bid_high_per_session.get('2', 0):.2f}"
mean_sem_bid_high_12_mois = f"{mean_bid_high_per_session.get('3', 0):.2f} ± {sem_bid_high_per_session.get('3', 0):.2f}"
mean_sem_bid_high_24_mois = f"{mean_bid_high_per_session.get('4', 0):.2f} ± {sem_bid_high_per_session.get('4', 0):.2f}"
print(sem_bid_high_per_session)


# Calculate the mean and SEM of bid for each session where calories = 'lo'
mean_bid_low_per_session = df[df['calories'] == 'lo'].groupby('session')['bid'].mean()
sem_bid_low_per_session = df[df['calories'] == 'lo'].groupby('session')['bid'].sem()

# Calculate the standard deviation of bid for each session where calories = 'lo'

std_bid_low_per_session = df[df['calories'] == 'lo'].groupby('session')['bid'].std()
print(sem_bid_low_per_session)

# Extract the mean and standard deviation of bid for each session where calories = 'lo'
mean_std_bid_low_pre_chirurgie = f"{mean_bid_low_per_session.get('1', 0):.2f} ± {std_bid_low_per_session.get('1', 0):.2f}"
mean_std_bid_low_4_mois = f"{mean_bid_low_per_session.get('2', 0):.2f} ± {std_bid_low_per_session.get('2', 0):.2f}"
mean_std_bid_low_12_mois = f"{mean_bid_low_per_session.get('3', 0):.2f} ± {std_bid_low_per_session.get('3', 0):.2f}"
mean_std_bid_low_24_mois = f"{mean_bid_low_per_session.get('4', 0):.2f} ± {std_bid_low_per_session.get('4', 0):.2f}"

# Extract the mean and SEM of bid for each session where calories = 'lo'
mean_sem_bid_low_pre_chirurgie = f"{mean_bid_low_per_session.get('1', 0):.2f} ± {sem_bid_low_per_session.get('1', 0):.2f}"
mean_sem_bid_low_4_mois = f"{mean_bid_low_per_session.get('2', 0):.2f} ± {sem_bid_low_per_session.get('2', 0):.2f}"
mean_sem_bid_low_12_mois = f"{mean_bid_low_per_session.get('3', 0):.2f} ± {sem_bid_low_per_session.get('3', 0):.2f}"
mean_sem_bid_low_24_mois = f"{mean_bid_low_per_session.get('4', 0):.2f} ± {sem_bid_low_per_session.get('4', 0):.2f}"

# Define the sessions
sessions = ['Pré-Chirurgie', '4 mois', '12 mois', '24 mois']

# Define the means and standard errors
mean_bid_low = [mean_bid_low_per_session.get(str(i), 0) for i in range(1, 5)]
sem_bid_low = [sem_bid_low_per_session.get(str(i), 0) for i in range(1, 5)]
mean_bid_high = [mean_bid_high_per_session.get(str(i), 0) for i in range(1, 5)]
sem_bid_high = [sem_bid_high_per_session.get(str(i), 0) for i in range(1, 5)]

# Plot the data
plt.figure(figsize=(10, 6))
plt.errorbar(sessions, mean_bid_low, yerr=sem_bid_low, label='WTP moyen faible', marker='o', capsize=5)
plt.errorbar(sessions, mean_bid_high, yerr=sem_bid_high, label='WTP moyen riche', marker='o', capsize=5)
plt.ylim(0, 5)
# Add labels for each point
for i, session in enumerate(sessions):
    if session == 'Pré-Chirurgie':
        plt.text(i-0.1, mean_bid_low[i] - sem_bid_low[i] + 0.2, f"{mean_bid_low[i]:.2f}".replace('.', ',') + " ± " + f"{sem_bid_low[i]:.2f}".replace('.', ','), ha='left', color='blue',fontsize=20)
        plt.text(i-0.1, mean_bid_high[i] - sem_bid_high[i] - 0.6, f"{mean_bid_high[i]:.2f}".replace('.', ',') + " ± " + f"{sem_bid_high[i]:.2f}".replace('.', ','), ha='left', color='orange',fontsize=20)
    else:
        plt.text(i, mean_bid_low[i] + sem_bid_low[i] + 0.1, f"{mean_bid_low[i]:.2f}".replace('.', ',') + " ± " + f"{sem_bid_low[i]:.2f}".replace('.', ','), ha='center', color='blue',fontsize=20)
        plt.text(i, mean_bid_high[i] + sem_bid_high[i] - 0.4, f"{mean_bid_high[i]:.2f}".replace('.', ',') + " ± " + f"{sem_bid_high[i]:.2f}".replace('.', ','), ha='center', color='orange',fontsize=20)

# Add labels and title
plt.ylabel('Valeur moyenne des mises ($)',fontsize=20)
plt.xticks(fontsize=20)

formatter = FuncFormatter(lambda x, _: f'{x:.2f}')
plt.yticks(fontsize=20)
plt.gca().tick_params(axis='x', pad=20)
plt.gca().yaxis.set_major_formatter(formatter)
plt.legend(['Faibles en calories', 'Riches en calories'], fontsize=20)
plt.grid(False)
plt.show()


# Add the mean and standard deviation of age to the results DataFrame
results.loc['N'] = [N_pre_chirurgie, N_4_mois, N_12_mois, N_24_mois, None]
results.loc['Age (mean ± std)'] = [mean_std_age_pre_chirurgie, mean_std_age_4_mois, mean_std_age_12_mois, mean_std_age_24_mois, None]
# Format the counts as "count of 0 : count of 1"
format_sexe_counts = lambda counts: f"{counts.get(0, 0)} : {counts.get(1, 0)}"
results.loc['Sexe (M:F)'] = [format_sexe_counts(sexe_pre_chirurgie), format_sexe_counts(sexe_4_mois), format_sexe_counts(sexe_12_mois), format_sexe_counts(sexe_24_mois), None]
# Add the gastrectomie counts to the results DataFrame
results.loc['Gastrectomie (n)'] = [gastrectomie_pre_chirurgie, gastrectomie_4_mois, gastrectomie_12_mois, gastrectomie_24_mois, None]
# Add the RYGB counts to the results DataFrame
results.loc['RYGB (n)'] = [rygb_pre_chirurgie, rygb_4_mois, rygb_12_mois, rygb_24_mois, None]

# Add the DBP-SD counts to the results DataFrame
results.loc['DBP-SD (n)'] = [dbp_sd_pre_chirurgie, dbp_sd_4_mois, dbp_sd_12_mois, dbp_sd_24_mois, None]

# Add the mean and standard deviation of imc_recherche to the results DataFrame
results.loc['IMC (kg/m2) (moy. ± et.)'] = [mean_std_imc_pre_chirurgie, mean_std_imc_4_mois, mean_std_imc_12_mois, mean_std_imc_24_mois, None]

# Add the mean and standard deviation of twl_recherche_x to the results DataFrame
results.loc['Pourcentage de perte de poids (%)'] = [mean_std_twl_pre_chirurgie, mean_std_twl_4_mois, mean_std_twl_12_mois, mean_std_twl_24_mois, None]

# Add the mean and standard deviation of bid to the results DataFrame
results.loc['Mise (moy. ± et.)'] = [mean_std_bid_pre_chirurgie, mean_std_bid_4_mois, mean_std_bid_12_mois, mean_std_bid_24_mois, None]

# Add the mean and standard deviation of bid to the results DataFrame
results.loc['Mise (élevé) (moy. ± et.)'] = [mean_std_bid_high_pre_chirurgie, mean_std_bid_high_4_mois, mean_std_bid_high_12_mois, mean_std_bid_high_24_mois, None]

# Add the mean and standard deviation of bid for low calories to the results DataFrame
results.loc['Mise (faible) (moy. ± et.)'] = [mean_std_bid_low_pre_chirurgie, mean_std_bid_low_4_mois, mean_std_bid_low_12_mois, mean_std_bid_low_24_mois, None]

print(results)


### Mixed model bid lo and session

In [ ]:
# Define the formula
formula = 'bid ~ session + age + type_chx + IMC_baseline + sexeF'

# Fit the mixed model
mixed_model = smf.mixedlm(formula, df_GI_bid_lo_pp_PYY, groups=df_GI_bid_lo_pp_PYY['id_participant'])
mixed_model_fit = mixed_model.fit()

# Residuals vs fitted values
residuals = mixed_model_fit.resid
fitted_values = mixed_model_fit.fittedvalues
sns.scatterplot(x=fitted_values, y=residuals)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted values')
plt.show()

# Print the summary of the model
print(mixed_model_fit.summary())

# QQ-plot of residuals
sm.qqplot(residuals, line='45')
plt.title('QQ-plot of Residuals')
plt.show()

# Breusch-Pagan test for homoscedasticity
bp_test = het_breuschpagan(mixed_model_fit.resid, mixed_model_fit.model.exog)
    
# Extract the p-value
bp_pvalue = bp_test[1]
print(f"Breusch-Pagan test p-value: {bp_pvalue}")
    
if bp_pvalue < 0.05:
    print("Heteroscedasticity detected. The variance of residuals is not constant.")
else:
    print("No heteroscedasticity detected. The variance of residuals is constant.")
    
# Shapiro-Wilk test for normality of residuals
shapiro_test = stats.shapiro(mixed_model_fit.resid)
print(f"Shapiro-Wilk test p-value: {shapiro_test.pvalue}")
    
if shapiro_test.pvalue < 0.05:
    print("Residuals are not normally distributed.")
else:
    print("Residuals are normally distributed.")

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = mixed_model_fit.model.exog_names
vif_data["VIF"] = [variance_inflation_factor(mixed_model_fit.model.exog, i) for i in range(mixed_model_fit.model.exog.shape[1])]
    
print(vif_data)

# Number of parameters in the model
k = len(mixed_model_fit.params)

# Number of observations
n = mixed_model_fit.nobs

# Log-likelihood of the model
log_likelihood = mixed_model_fit.llf

# Calculate AIC
aic = 2 * k - 2 * log_likelihood

# Calculate BIC
bic = np.log(n) * k - 2 * log_likelihood

print(f"AIC: {aic}")
print(f"BIC: {bic}")


# Correlations

## Correlations sessions combined and ghrel tot

In [ ]:
from scipy.stats import pearsonr


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, spearmanr, normaltest

# Create scatter plots
plt.figure(figsize=(15, 5))

# Filter the data for each type_chx
type_chx_1 = df_GI_bid_hi_ac_Ghrelin_Totale[df_GI_bid_hi_ac_Ghrelin_Totale['type_chx'] == 1]
type_chx_2 = df_GI_bid_hi_ac_Ghrelin_Totale[df_GI_bid_hi_ac_Ghrelin_Totale['type_chx'] == 2]
type_chx_3 = df_GI_bid_hi_ac_Ghrelin_Totale[df_GI_bid_hi_ac_Ghrelin_Totale['type_chx'] == 3]

def compute_correlation(data):
    if len(data) >= 8:
        # Test for normality
        stat_bid, p_bid = normaltest(data['bid'])
        stat_ghrelin, p_ghrelin = normaltest(data['Ghrelin_Totale'])
        
        if p_bid > 0.05 and p_ghrelin > 0.05:
            # Data is normally distributed, use Pearson correlation
            r, p = pearsonr(data['bid'], data['Ghrelin_Totale'])
            corr_type = 'Pearson'
        else:
            # Data is not normally distributed, use Spearman correlation
            r, p = spearmanr(data['bid'], data['Ghrelin_Totale'])
            corr_type = 'Spearman'
    else:
        # Not enough samples, use Spearman correlation
        r, p = spearmanr(data['bid'], data['Ghrelin_Totale'])
        corr_type = 'Spearman'
    return r, p, corr_type

# Compute correlation and p-value for each type_chx
r1, p1, corr_type1 = compute_correlation(type_chx_1)
r2, p2, corr_type2 = compute_correlation(type_chx_2)
r3, p3, corr_type3 = compute_correlation(type_chx_3)

plt.subplot(1, 3, 1)
sns.scatterplot(data=type_chx_1, x='bid', y='Ghrelin_Totale')
plt.title(f'Type_chx 1\n{corr_type1} r={r1:.2f}, p={p1:.4f}')

plt.subplot(1, 3, 2)
sns.scatterplot(data=type_chx_2, x='bid', y='Ghrelin_Totale')
plt.title(f'Type_chx 2\n{corr_type2} r={r2:.2f}, p={p2:.4f}')

plt.subplot(1, 3, 3)
sns.scatterplot(data=type_chx_3, x='bid', y='Ghrelin_Totale')
plt.title(f'Type_chx 3\n{corr_type3} r={r3:.2f}, p={p3:.4f}')

plt.tight_layout()
plt.show()

## Correlations all sessions and ghrel Tot

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, spearmanr, normaltest

# Create scatter plots
plt.figure(figsize=(15, 20))

# Filter the data for each type_chx and session
sessions = ['1', '2', '3', '4']
type_chx_values = [1, 2, 3]

for i, session in enumerate(sessions):
    for j, type_chx in enumerate(type_chx_values):
        data = df_GI_bid_hi_ac_Ghrelin_Totale[(df_GI_bid_hi_ac_Ghrelin_Totale['session'] == session) & (df_GI_bid_hi_ac_Ghrelin_Totale['type_chx'] == type_chx)]
        if not data.empty:
            if len(data) >= 8:
                # Test for normality
                stat_bid, p_bid = normaltest(data['bid'])
                stat_ghrelin, p_ghrelin = normaltest(data['Ghrelin_Totale'])
                
                if p_bid > 0.05 and p_ghrelin > 0.05:
                    # Data is normally distributed, use Pearson correlation
                    r, p = pearsonr(data['bid'], data['Ghrelin_Totale'])
                    corr_type = 'Pearson'
                else:
                    # Data is not normally distributed, use Spearman correlation
                    r, p = spearmanr(data['bid'], data['Ghrelin_Totale'])
                    corr_type = 'Spearman'
            else:
                # Not enough samples, use Spearman correlation
                r, p = spearmanr(data['bid'], data['Ghrelin_Totale'])
                corr_type = 'Spearman'
        else:
            r, p = float('nan'), float('nan')
            corr_type = 'None'
        
        plt.subplot(4, 3, i * 3 + j + 1)
        sns.scatterplot(data=data, x='bid', y='Ghrelin_Totale')
        plt.title(f'Session {session} Type_chx {type_chx}\n{corr_type} r={r:.2f}, p={p:.4f}')

plt.tight_layout()
plt.show()